In [85]:
from bs4 import BeautifulSoup
import requests
import traceback
from tqdm import tqdm
import json
from collections import Counter
from pprint import pprint

In [39]:
# format of text file looks like:
"""
CSE         1244 (Summer 2024)         updated: 26-Feb-2024

                       class#    (autoenrolls)                                enrld/limit/+wait

     CSE 1110             5465 L                                      ONLINE       5/40       {8W1} M.Mallon

     CSE 1222             5466 L                 M W      0935A-1055  BE0188       5/40       {1S} L.Madrid
     CSE 1222             5467 B  ( 5466)           R     0935A-1055  BE0310       5/40       {1S} L.Madrid

     CSE 1223             8369 L                 M W      1245P-0205  CL0120       3/40       {1S} M.Mallon
     CSE 1223             8368 B  ( 8369)           R     1245P-0205  BE0470       3/40       {1S} M.Mallon
"""


'\nCSE         1244 (Summer 2024)         updated: 26-Feb-2024\n\n                       class#    (autoenrolls)                                enrld/limit/+wait\n\n     CSE 1110             5465 L                                      ONLINE       5/40       {8W1} M.Mallon\n\n     CSE 1222             5466 L                 M W      0935A-1055  BE0188       5/40       {1S} L.Madrid\n     CSE 1222             5467 B  ( 5466)           R     0935A-1055  BE0310       5/40       {1S} L.Madrid\n\n     CSE 1223             8369 L                 M W      1245P-0205  CL0120       3/40       {1S} M.Mallon\n     CSE 1223             8368 B  ( 8369)           R     1245P-0205  BE0470       3/40       {1S} M.Mallon\n'

In [40]:
# parse through the file. the first few lines are metadata and headers, then the rest are classes
# there are 8 columns: course info: {department, number}, course unique id: {section/class #}, {type} (e.g. L or B), {autoenrolls}, {days abbreviated as letters}, {time}, {location}, {enrolled/limit/+wait}, {instructor}
# completely separate classes are separated by a newline, two sections of the same class are separated in adjacent lines
# we want to extract the course info, section/class #, type, days, time, location, enrolled/limit/+wait, and instructor
# we want to output the data in a csv file
# the csv file should have the following columns: {department, number}, {section/class #}, {type}, {autoenrolls}, {days}, {time}, {location}, {enrolled}, {limit}, {+wait}, {instructor}
# the csv file should be named "classes.csv"

input_text_file = "1248.txt"
output_csv_file = "classes.csv"

with open(input_text_file, "r") as file:
    lines = file.readlines()
    # store first few lines as metadata
    metadata = lines[:3]
    # store the headers
    headers = lines[2].split()
    # store the classes from line 4 until we reach a line that begins with "INDependent study classes"
    classes = []
    for line in lines[4:]:
        if line.startswith("IND"):
            break
        classes.append(line)
    print(classes)

# remove leading and trailing whitespace
metadata = [line.strip() for line in metadata]
headers = [header.strip() for header in headers]
classes = [class_.strip() for class_ in classes]

print(metadata)
print(headers)
print(classes)

['     CSE 1110             6562 L                                      ONLINE       0/150      \n', '\n', '     CSE 1111             6563 L                  T R     0935A-1055  CL0220       0/40       \n', '     CSE 1111             6564 B  ( 6563)        M        1020A       DL0280       0/40       \n', '\n', '     CSE 1111             6565 L                  T R     1110A-1230  BE0140       0/40       \n', '     CSE 1111             6566 B  ( 6565)        M        1130A       DL0280       0/40       \n', '\n', '     CSE 1111             6567 L                  T R     0220P-0340  CL0220       0/40       \n', '     CSE 1111             6568 B  ( 6567)        M        0150P       DL0280       0/40       \n', '\n', '     CSE 1222             6930 L                  T R     0910A       BE0144       0/70       \n', '     CSE 1222             6931 B  ( 6930)        M        0800A       BE0480       0/40       \n', '     CSE 1222             7268 B  ( 6930)        M        0910A       BE04

In [41]:
# parse through the classes
parsed_classes = []
for class_ in classes:
    # split the class into its columns
    columns = class_.split()
    # if the columns are empty, then we have a new class
    if not columns:
        parsed_classes.append([])
    else:
        if len(parsed_classes) == 0:
            parsed_classes.append([])
        parsed_classes[-1].append(columns)
    
print(parsed_classes)

[[['CSE', '1110', '6562', 'L', 'ONLINE', '0/150']], [['CSE', '1111', '6563', 'L', 'T', 'R', '0935A-1055', 'CL0220', '0/40'], ['CSE', '1111', '6564', 'B', '(', '6563)', 'M', '1020A', 'DL0280', '0/40']], [['CSE', '1111', '6565', 'L', 'T', 'R', '1110A-1230', 'BE0140', '0/40'], ['CSE', '1111', '6566', 'B', '(', '6565)', 'M', '1130A', 'DL0280', '0/40']], [['CSE', '1111', '6567', 'L', 'T', 'R', '0220P-0340', 'CL0220', '0/40'], ['CSE', '1111', '6568', 'B', '(', '6567)', 'M', '0150P', 'DL0280', '0/40']], [['CSE', '1222', '6930', 'L', 'T', 'R', '0910A', 'BE0144', '0/70'], ['CSE', '1222', '6931', 'B', '(', '6930)', 'M', '0800A', 'BE0480', '0/40'], ['CSE', '1222', '7268', 'B', '(', '6930)', 'M', '0910A', 'BE0480', '0/40']], [['CSE', '1222', '6921', 'L', 'T', 'R', '1130A', 'DL0113', '0/160'], ['CSE', '1222', '6922', 'B', '(', '6921)', 'M', '1020A', 'BE0310', '0/40'], ['CSE', '1222', '10783', 'B', '(', '6921)', 'M', '1020A', 'BE0310', '0/40'], ['CSE', '1222', '7269', 'B', '(', '6921)', 'M', '1130A'

In [42]:
# parse through the parsed classes and continue to process the data to properly format it to be written to a csv file
# What I currently have: department, number, section/class #, type (e.g. L or B or others), autoenrolls (not always there), days (currently across multiple elements), time (not always present due to online classes), location, enrolled, limit, +wait (not always there), duration (not always present), instructor (not always present)
# I need to combine the days into a single element, and I need to account for the fact that some elements may not be present
# I also need to account for the fact that some classes have multiple sections

# I will create a list of dictionaries, where each dictionary represents a class
# I will then write the data to a csv file

# create a list of dictionaries
classes = []
for parsed_class in parsed_classes:
    course_info = []
    for class_ in parsed_class:
        class_info = {}
        class_copy = class_.copy()
        
        # line could begin with "and". if it does, add the entire line to the previous class_info dictionary as a "additional information" key
        if class_copy[0] == "and":
            class_copy.pop(0)
            course_info[-1]["additional information"] = " ".join(class_copy) # join the rest of the elements into a single string and add it to the dictionary
            continue

        # extract the department and number, then remove them from the list
        department, number = class_copy[0], class_copy[1]
        class_info["department"] = department
        class_info["number"] = number
        class_copy.pop(0)
        class_copy.pop(0)

        # extract the campus, section/class #, type, autoenrolls, days, time, location, enrolled, limit, +wait, duration, and instructor
        # campus is not always present, but if it is, it is the next element
        campus = ""
        if class_copy[0].isalpha():
            campus = class_copy[0]
            class_copy.pop(0)

        # section/class # and type are the next two elements
        section_class_number = class_copy[0]
        type_ = class_copy[1]
        class_copy.pop(0)
        class_copy.pop(0)

        # autoenrolls is not always present, but if it is, it is currently stored across two elements: element 4: "(", element 5: "{autoenrolls})" or element 4: "(autoenrolls)"
        # I want only the {autoenrolls} part
        autoenrolls = ""
        if class_copy[0].startswith("("):
            if class_copy[0] == "(":
                class_copy.pop(0)
                autoenrolls = class_copy[0].strip(")")
                class_copy.pop(0)
            else:
                autoenrolls = class_copy[0].strip("()")
                class_copy.pop(0)

        
        # days is currently stored across multiple elements, but might not be present (e.g. ONLINE classes do not have days or times)
        # combine the days into a single element (have to check if the days are present)
        days = ""
        # keep adding elements to days until we reach the time
        while class_copy[0][0] in ["M", "T", "W", "R", "F", "S", "U"]:
            days += class_copy[0]
            class_copy.pop(0)
        
        # time is not always present, but if it is, it starts with a digit
        time = ""
        if class_copy[0][0].isdigit() and "/" not in class_copy[0]:
            time = class_copy[0]
            class_copy.pop(0)
        
        # location is the next element. not always present, but if it is present, it is the next element
        location = ""
        if class_copy[0][0].isalpha():
            location = class_copy[0]
            class_copy.pop(0)

        # enrolled, limit, are stored across a single element as {enrolled}/{limit}. +wait is stored in the element after enrolled and limit, but +wait is not always present
        print(class_copy[0])
        enrolled, limit = class_copy[0].split("/")
        class_copy.pop(0)
        plus_wait = ""
        if len(class_copy) > 0 and class_copy[0].startswith("+"):
            plus_wait = class_copy[0]
            class_copy.pop(0)
        
        # duration is not always present, but if it is, it is the next element and startswith "{"
        duration = ""
        if len(class_copy) > 0 and class_copy[0].startswith("{"):
            duration = class_copy[0]
            class_copy.pop(0)
        
        # instructor is the next element, but is not always present
        instructor = ""
        # print(class_copy[0])
        if len(class_copy) > 0 and class_copy[0].isalpha():
            instructor = class_copy[0]
            class_copy.pop(0)
        
        # final descriptor of instructor is the next element, but is not always present (starts with "(")
        instructor_descriptor = ""
        if len(class_copy) > 0 and class_copy[0].startswith("("):
            instructor_descriptor = class_copy[0]
            class_copy.pop(0)
        
        # add the class to the list
        class_info["section/class #"] = section_class_number
        class_info["type"] = type_
        class_info["autoenrolls"] = autoenrolls
        class_info["days"] = days
        class_info["time"] = time
        class_info["location"] = location
        class_info["enrolled"] = enrolled
        class_info["limit"] = limit
        class_info["+wait"] = plus_wait
        class_info["duration"] = duration
        class_info["instructor"] = instructor
        class_info["instructor descriptor"] = instructor_descriptor
        course_info.append(class_info)
    classes.append(course_info)

print(classes)
        








0/150
0/40
0/40
0/40
0/40
0/40
0/40
0/70
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/80
0/80
0/40
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/151
0/80
0/40
0/40
0/0
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/34
0/40
0/40
0/40
0/32
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/45
0/45
0/45
0/60
0/60
0/45
0/45
0/40
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/60
0/60
0/60
0/45
0/45
0/45
0/45
0/40
0/40
0/30
0/30
0/30
0/30
0/30
0

In [43]:
# write the data to a json file
with open("testcourses-1248.json", "w") as f:
    json.dump(classes, f)

In [24]:
def split_class_file(content):
    lines = content.split("\n")
    # store first few lines as metadata
    # metadata goes from beginning of file to the line before the headers line which looks like "class#    (autoenrolls)                                enrld/limit/+wait"
    header_index = 0
    for i, line in enumerate(lines):
        if "class#" in line and "autoenrolls" in line and "enrld/limit/+wait" in line:
            header_index = i
            break
    metadata = lines[:header_index]
    # store the headers
    headers = lines[header_index].split()
    # store the classes from line header_index+2 until we reach a line that begins with "INDependent study classes" or "waitlist"
    classes = []
    for line in lines[header_index+2:]:
        if line.startswith("IND") or line.lower().startswith("waitlist"):
            # continue to pop the last element until we reach a non-empty line
            while len(classes) > 0 and len(classes[-1]) == 0:
                classes.pop()
            break
        classes.append(line)
    # print(classes)

    # remove leading and trailing whitespace
    metadata = [line.strip() for line in metadata]
    headers = [header.strip() for header in headers]
    classes = [class_.strip() for class_ in classes]

    return metadata, headers, classes

In [25]:
def preprocess_classes(classes):
    # parse through the classes
    parsed_classes = []
    for class_ in classes:
        # split the class into its columns
        columns = class_.split()
        # if the columns are empty, then we have a new class
        if not columns:
            parsed_classes.append([])
        else:
            if len(parsed_classes) == 0:
                parsed_classes.append([])
            parsed_classes[-1].append(columns)
        
    return parsed_classes

In [33]:
def parse_classes(parsed_classes):
    # parse through the parsed classes and continue to process the data to properly format it to be written to a csv file
    # What I currently have: department, number, section/class #, type (e.g. L or B or others), autoenrolls (not always there), days (currently across multiple elements), time (not always present due to online classes), location, enrolled, limit, +wait (not always there), duration (not always present), instructor (not always present)
    # I need to combine the days into a single element, and I need to account for the fact that some elements may not be present
    # I also need to account for the fact that some classes have multiple sections

    # I will create a list of dictionaries, where each dictionary represents a class
    # I will then write the data to a csv file

    # create a list of dictionaries
    
    try:
        classes = []
        for parsed_class in parsed_classes:
            course_info = []
            for class_ in parsed_class:
                class_info = {}
                class_copy = class_.copy()
                
                # line could begin with "and". if it does, add the entire line to the previous class_info dictionary as a "additional information" key
                if class_copy[0] == "and":
                    class_copy.pop(0)
                    course_info[-1]["additional information"] = " ".join(class_copy) # join the rest of the elements into a single string and add it to the dictionary
                    continue

                # extract the department and number, then remove them from the list
                department, number = class_copy[0], class_copy[1]
                class_info["department"] = department
                class_info["number"] = number
                class_copy.pop(0)
                class_copy.pop(0)

                # extract the campus, section/class #, type, autoenrolls, days, time, location, enrolled, limit, +wait, duration, and instructor
                # campus is not always present, but if it is, it is the next element
                campus = ""
                if class_copy[0].isalpha():
                    campus = class_copy[0]
                    class_copy.pop(0)

                # section/class # and type are the next two elements
                section_class_number = class_copy[0]
                type_ = class_copy[1]
                class_copy.pop(0)
                class_copy.pop(0)

                # autoenrolls is not always present, but if it is, it is currently stored across two elements: element 4: "(", element 5: "{autoenrolls})" or element 4: "(autoenrolls)"
                # I want only the {autoenrolls} part
                autoenrolls = ""
                if class_copy[0].startswith("("):
                    if class_copy[0] == "(":
                        class_copy.pop(0)
                        autoenrolls = class_copy[0].strip(")")
                        class_copy.pop(0)
                    else:
                        autoenrolls = class_copy[0].strip("()")
                        class_copy.pop(0)

                
                # days is currently stored across multiple elements, but might not be present (e.g. ONLINE classes do not have days or times)
                # combine the days into a single element (have to check if the days are present)
                days = ""
                # keep adding elements to days until we reach the time
                while class_copy[0][0].upper() in ["M", "T", "W", "R", "F", "S", "U"]:
                    days += class_copy[0]
                    class_copy.pop(0)
                
                # time is not always present, but if it is, it starts with a digit
                time = ""
                if class_copy[0][0].isdigit() and "/" not in class_copy[0]:
                    time = class_copy[0]
                    class_copy.pop(0)
                
                # location is the next element. not always present, but if it is present, it is the next element
                location = ""
                if class_copy[0][0].isalpha():
                    location = class_copy[0]
                    class_copy.pop(0)

                # enrolled, limit, are stored across a single element as {enrolled}/{limit}. +wait is stored in the element after enrolled and limit, but +wait is not always present
                # print(class_copy[0])
                enrolled, limit = class_copy[0].split("/")
                class_copy.pop(0)
                plus_wait = ""
                if len(class_copy) > 0 and class_copy[0].startswith("+"):
                    plus_wait = class_copy[0]
                    class_copy.pop(0)
                
                # duration is not always present, but if it is, it is the next element and startswith "{"
                duration = ""
                if len(class_copy) > 0 and class_copy[0].startswith("{"):
                    duration = class_copy[0]
                    class_copy.pop(0)
                
                # instructor is the next element, but is not always present
                instructors = []
                # print(class_copy[0])
                while len(class_copy) > 0 and class_copy[0][0].isalpha():
                    instructors.append(class_copy[0].strip(","))
                    class_copy.pop(0)
                
                # final descriptor of instructor is the next element, but is not always present (starts with "(")
                instructor_descriptor = ""
                if len(class_copy) > 0 and class_copy[0].startswith("("):
                    instructor_descriptor = class_copy[0]
                    class_copy.pop(0)
                
                # add the class to the list
                class_info["section/class #"] = section_class_number
                class_info["type"] = type_
                class_info["autoenrolls"] = autoenrolls
                class_info["days"] = days
                class_info["time"] = time
                class_info["location"] = location
                class_info["enrolled"] = enrolled
                class_info["limit"] = limit
                class_info["+wait"] = plus_wait
                class_info["duration"] = duration
                class_info["instructor"] = instructors
                class_info["instructor descriptor"] = instructor_descriptor
                course_info.append(class_info)
            classes.append(course_info)
    except Exception as e:
        # return the last class that was being processed
        return "Error", traceback.format_exc(), class_

    return "Success", "OK", classes

In [4]:
# prev_department_short_names = getDepartmentShortNames(1248)

prev_department_short_names_SP24 = ['ACADAFF', 'ACCTMIS', 'ACCAD', 'AEROENG', 'AFAMAST', 'AEDECON', 'ACEL', 'AGRCOMM', 'AGSYSMT', 'ASE', 'AIRSCI', 'AMINSTS', 'ASL', 'ANATOMY', 'ANIMSCI', 'ANMLTEC', 'ANTHROP', 'ARABIC', 'ARCH', 'ART', 'ARTEDUC', 'ARTSSCI', 'ASAMSTS', 'ASTRON', 'ATHTRNG', 'ATMOSSC', 'AVIATN', 'BIOCHEM', 'BIOETHC', 'BIOPHRM', 'BIOLOGY', 'BIOMEDE', 'BMI', 'BIOMSCI', 'BSGP', 'BIOPHYS', 'BIOTECH', 'BCS', 'BUSOBA', 'BUSFIN', 'BUSMHR', 'BUSML', 'BUSADM', 'BUSTEC', 'CONSRTM', 'CBG', 'CATALAN', 'CLLC', 'CHEMPHY', 'CBE', 'CHEM', 'CHINESE', 'CRPLAN', 'CIVILEN', 'CLAS', 'MCR', 'COMM', 'COMLDR', 'COMPSTD', 'CSE', 'CONSYSM', 'CSCFFS', 'CSFRST', 'CSHSPMG', 'CONSCI', 'CRPSOIL', 'CSTW', 'CZECH', 'DANCE', 'DENTHYG', 'DENT', 'DESIGN', 'DSABLST', 'DNE', 'EARTHSC', 'EALL', 'EEURLL', 'ECON', 'ESCE', 'ESCFE', 'ESEPSY', 'ESEADM', 'ESEPOL', 'ESHESA', 'ESLTECH', 'ESPHE', 'ESQUAL', 'ESQREM', 'ESSPSY', 'ESSPED', 'ESWDE', 'EHE', 'EDUTL', 'EDUCST', 'ECE', 'ENGR', 'ENGREDU', 'ENGTECH', 'ENGRTEC', 'ENGLISH', 'ENTMLGY', 'ENR', 'ENVENG', 'ENVSCI', 'ETHNSTD', 'EEOB', 'EXP', 'FILMSTD', 'FDSCTE', 'FABENG', 'FAES', 'FRENCH', 'FRIT', 'GENED', 'GENSTDS', 'GENBIOL', 'GENCHEM', 'GENCOMM', 'GENMATH', 'GENSSC', 'GEOSCIM', 'GEOG', 'GERMAN', 'GRADSCH', 'GREEK', 'HCINNOV', 'HIMS', 'HTHRHSC', 'HW', 'HWIH', 'HEBREW', 'HECCREG', 'HINDI', 'HISTORY', 'HISTART', 'HONORS', 'HORTTEC', 'HCS', 'HDFS', 'HUMNNTR', 'ISE', 'INTMED', 'INTSTDS', 'ISLAM', 'ITALIAN', 'JAPANSE', 'JEWSHST', 'KINESIO', 'KNHES', 'KNPE', 'KNSFHP', 'KNSISM', 'KNOW', 'KOREAN', 'LARCH', 'LATIN', 'LAW', 'LING', 'MBA', 'MDN', 'MATSCEN', 'MATH', 'MEATSCI', 'MECHENG', 'MEDDIET', 'MEDLBS', 'MEDCOLL', 'MEDREN', 'MEDMCIM', 'MICRBIO', 'MILSCI', 'MDRNGRK', 'MCDBIO', 'MOLGEN', 'MVNGIMG', 'MUSIC', 'NAVALSC', 'NELC', 'NEURSGY', 'NEURO', 'NEUROSC', 'NEUROGS', 'NUCLREN', 'NURSING', 'NRSADVN', 'NRSPRCT', 'OCCTHER', 'OSBP', 'OPTOM', 'OTOLARN', 'PATHOL', 'PEDS', 'PERSIAN', 'PHR', 'PHILOS', 'PHYSMED', 'PHYSTHR', 'PHYSICS', 'PHYSIO', 'PLNTPTH', 'POLISH', 'POLITSC', 'PORTGSE', 'PSYBHLH', 'PSYCH', 'PUBHBIO', 'PUBHEHS', 'PUBHEPI', 'PUBHHBP', 'PUBHHMP', 'PUBAFRS', 'PUBHLTH', 'QUECHUA', 'ROOM', 'RADONC', 'RADSCI', 'RADIOLG', 'RELSTDS', 'RESPTHR', 'ROMLING', 'ROMANIA', 'RURLSOC', 'RUSSIAN', 'SANSKRT', 'SCANDVN', 'SLAVIC', 'SOCWORK', 'SOCIOL', 'SOMALI', 'SASIA', 'SPANISH', 'SPHHRNG', 'STAT', 'SURGERY', 'SWAHILI', 'SWEDISH', 'TECPHYS', 'THEATRE', 'GRADTDA', 'TURKISH', 'URDU', 'UROLOGY', 'UZBEK', 'VETBIOS', 'VETCLIN', 'VMCOLL', 'VETPREV', 'VISSCI', 'WELDENG', 'WGSST', 'YIDDISH']
prev_department_short_names_AU24 = ['ACADAFF', 'ACCTMIS', 'ACCAD', 'AEROENG', 'AFAMAST', 'AEDECON', 'ACEL', 'AGRCOMM', 'AGSYSMT', 'ASE', 'AIRSCI', 'AMINSTS', 'ASL', 'ANATOMY', 'ANIMSCI', 'ANMLTEC', 'ANTHROP', 'ARABIC', 'ARCH', 'ART', 'ARTEDUC', 'ARTSSCI', 'ASAMSTS', 'ASTRON', 'ATHTRNG', 'ATMOSSC', 'AVIATN', 'BIOCHEM', 'BIOETHC', 'BIOPHRM', 'BIOLOGY', 'BMEA', 'BIOMEDE', 'BMI', 'BIOMSCI', 'BSGP', 'BIOPHYS', 'BIOTECH', 'BCS', 'BUSOBA', 'BUSFIN', 'BUSMHR', 'BUSML', 'BUSADM', 'BUSTEC', 'CONSRTM', 'CBG', 'CATALAN', 'CLLC', 'CHEMPHY', 'CBE', 'CHEM', 'CHINESE', 'CRPLAN', 'CIVILEN', 'CLAS', 'MCR', 'COMM', 'COMLDR', 'COMPSTD', 'CSE', 'CONSYSM', 'CSCFFS', 'CSFRST', 'CSHSPMG', 'CONSCI', 'CRPSOIL', 'CZECH', 'DANCE', 'DENTHYG', 'DENT', 'DESIGN', 'DSABLST', 'DNE', 'EARTHSC', 'EALL', 'ECON', 'ESCPSY', 'ESCE', 'ESCFE', 'ESEPSY', 'ESEADM', 'ESEPOL', 'ESHESA', 'ESLTECH', 'ESPHE', 'ESQUAL', 'ESQREM', 'ESSPSY', 'ESSPED', 'ESWDE', 'EHE', 'EDUTL', 'EDUCST', 'ECE', 'ENGR', 'ENGREDU', 'ENGTECH', 'ENGRTEC', 'ENGLISH', 'ENTMLGY', 'ENR', 'ENVENG', 'ENVSCI', 'ENVSCT', 'ETHNSTD', 'EEOB', 'EXP', 'FILMSTD', 'FDSCTE', 'FABENG', 'FAES', 'FRENCH', 'FRIT', 'GENED', 'GENSTDS', 'GENBIOL', 'GENCHEM', 'GENCOMM', 'GENMATH', 'GENSSC', 'GEOSCIM', 'GEOG', 'GERMAN', 'GRADSCH', 'GREEK', 'HCINNOV', 'HIMS', 'HTHRHSC', 'HW', 'HEBREW', 'HECCREG', 'HINDI', 'HISTORY', 'HISTART', 'HONORS', 'HORTTEC', 'HCS', 'HDFS', 'HUMNNTR', 'ISE', 'INTMED', 'INTSTDS', 'ISLAM', 'ITALIAN', 'JAPANSE', 'JEWSHST', 'KINESIO', 'KNHES', 'KNPE', 'KNSFHP', 'KNSISM', 'KNOW', 'KOREAN', 'LARCH', 'LATIN', 'LAW', 'LING', 'MBA', 'MDN', 'MATSCEN', 'MATH', 'MEATSCI', 'MECHENG', 'MEDDIET', 'MEDLBS', 'MEDCOLL', 'MEDREN', 'MEDMCIM', 'MICRBIO', 'MILSCI', 'MDRNGRK', 'MCDBIO', 'MOLGEN', 'MVNGIMG', 'MUSIC', 'NAVALSC', 'NELC', 'NEURSGY', 'NEURO', 'NEUROSC', 'NEUROGS', 'NUCLREN', 'NURSING', 'NRSADVN', 'NRSPRCT', 'OCCTHER', 'OSBP', 'OPTOM', 'OTOLARN', 'PATHOL', 'PEDS', 'PERSIAN', 'PHR', 'PHILOS', 'PHYSTHR', 'PHYSICS', 'PHYSIO', 'PLNTPTH', 'POLISH', 'POLITSC', 'PORTGSE', 'PSYBHLH', 'PSYCH', 'PUBHBIO', 'PUBHEHS', 'PUBHEPI', 'PUBHHBP', 'PUBHHMP', 'PUBAFRS', 'PUBHLTH', 'QUECHUA', 'ROOM', 'RADONC', 'RADSCI', 'RADIOLG', 'RELSTDS', 'RESPTHR', 'ROMANIA', 'RURLSOC', 'RUSSIAN', 'SANSKRT', 'SCANDVN', 'SCHOLAR', 'SLAVIC', 'SOCWORK', 'SOCIOL', 'SOMALI', 'SASIA', 'SPANISH', 'SPHHRNG', 'STAT', 'SURGERY', 'SWAHILI', 'SWEDISH', 'THEATRE', 'GRADTDA', 'TURKISH', 'URDU', 'UROLOGY', 'UZBEK', 'VETBIOS', 'VETCLIN', 'VMCOLL', 'VETPREV', 'VISSCI', 'WELDENG', 'WGSST', 'YIDDISH']
prev_department_short_names_SU24 = ['ACCTMIS', 'ACCAD', 'AEROENG', 'AFAMAST', 'AEDECON', 'ACEL', 'AGRCOMM', 'AGSYSMT', 'ASE', 'AMINSTS', 'ASL', 'ANATOMY', 'ANIMSCI', 'ANMLTEC', 'ANTHROP', 'ARABIC', 'ARCH', 'ART', 'ARTEDUC', 'ARTSSCI', 'ASAMSTS', 'ASTRON', 'ATHTRNG', 'ATMOSSC', 'AVIATN', 'BIOCHEM', 'BIOETHC', 'BIOPHRM', 'BIOLOGY', 'BIOMEDE', 'BMI', 'BSGP', 'BIOPHYS', 'BUSOBA', 'BUSFIN', 'BUSMHR', 'BUSML', 'BUSADM', 'BUSTEC', 'CONSRTM', 'CBG', 'CLLC', 'CHEMPHY', 'CBE', 'CHEM', 'CHINESE', 'CRPLAN', 'CIVILEN', 'CLAS', 'MCR', 'COMM', 'COMLDR', 'COMPSTD', 'CSE', 'CONSYSM', 'CSCFFS', 'CSFRST', 'CSHSPMG', 'CONSCI', 'CRPSOIL', 'DANCE', 'DENTHYG', 'DENT', 'DESIGN', 'DSABLST', 'DNE', 'EARTHSC', 'EALL', 'ECON', 'ESCE', 'ESEPSY', 'ESEADM', 'ESEPOL', 'ESHESA', 'ESLTECH', 'ESPHE', 'ESQUAL', 'ESQREM', 'ESSPSY', 'ESSPED', 'ESWDE', 'EHE', 'EDUTL', 'EDUCST', 'ECE', 'ENGR', 'ENGREDU', 'ENGTECH', 'ENGLISH', 'ENTMLGY', 'ENR', 'ENVENG', 'EEOB', 'EXP', 'FILMSTD', 'FDSCTE', 'FABENG', 'FAES', 'FRENCH', 'FRIT', 'GENED', 'GEOSCIM', 'GEOG', 'GERMAN', 'GRADSCH', 'GREEK', 'HCINNOV', 'HIMS', 'HTHRHSC', 'HW', 'HISTORY', 'HISTART', 'HORTTEC', 'HCS', 'HDFS', 'HUMNNTR', 'ISE', 'INTMED', 'INTSTDS', 'ITALIAN', 'JAPANSE', 'KINESIO', 'KNHES', 'KNPE', 'KNSFHP', 'KNSISM', 'KNOW', 'KOREAN', 'LARCH', 'LATIN', 'LAW', 'LING', 'MBA', 'MDN', 'MATSCEN', 'MATH', 'MEATSCI', 'MECHENG', 'MEDDIET', 'MEDLBS', 'MEDCOLL', 'MEDREN', 'MEDMCIM', 'MICRBIO', 'MCDBIO', 'MOLGEN', 'MVNGIMG', 'MUSIC', 'NELC', 'NEURSGY', 'NEURO', 'NEUROSC', 'NUCLREN', 'NURSING', 'NRSADVN', 'NRSPRCT', 'ORIENTAT', 'OCCTHER', 'OSBP', 'OPTOM', 'OTOLARN', 'PATHOL', 'PEDS', 'PERSIAN', 'PHR', 'PHILOS', 'PHYSTHR', 'PHYSICS', 'PHYSIO', 'PLNTPTH', 'POLITSC', 'PORTGSE', 'PSYBHLH', 'PSYCH', 'PUBHBIO', 'PUBHEHS', 'PUBHEPI', 'PUBHHBP', 'PUBHHMP', 'PUBAFRS', 'PUBHLTH', 'RADONC', 'RADSCI', 'RESPTHR', 'RURLSOC', 'RUSSIAN', 'SLAVIC', 'SOCWORK', 'SOCIOL', 'SPANISH', 'SPHHRNG', 'STAT', 'SURGERY', 'SWAHILI', 'THEATRE', 'GRADTDA', 'VETBIOS', 'VETCLIN', 'VMCOLL', 'VETPREV', 'VISSCI', 'WELDENG', 'WGSST']
prev_department_short_names_combined = set(prev_department_short_names_SP24 + prev_department_short_names_AU24 + prev_department_short_names_SU24) # eventually replace with department short names from database

# scrape through https://www.asc.ohio-state.edu/barrett.3/schedule/{department_short_name}/{term}.txt
dept_term_url = "https://www.asc.ohio-state.edu/barrett.3/schedule/{}/{}.txt"

# test_dept = ["CSE"] # replace later with prev_department_short_names_combined
# test_term = ["1248"] # replace later with [1242, 1244, 1248]

terms = [1242, 1244, 1248]
all_classes_by_term_department = {
    1242: {},
    1244: {},
    1248: {}
}


In [ ]:
for dept in tqdm(prev_department_short_names_combined):
    for term in terms:
        print("Currently on department: {} and term: {}".format(dept, term))
        url = dept_term_url.format(dept, term)
        page = requests.get(url)
        # print(page.text)

        if page.status_code != 200:
            all_classes_by_term_department[term][dept] = []
            continue

        metadata, headers, classes = split_class_file(page.text)
        preprocessed_classes = preprocess_classes(classes)
        indicator, stacktrace, parsed_classes = parse_classes(preprocessed_classes)
        if indicator == "Error":
            print("Error occurred while parsing the file for department: {} and term: {}".format(dept, term))
            all_classes_by_term_department[term][dept] = {
                "error": stacktrace,
                "last_class": parsed_classes
            }
            continue
        else:
            all_classes_by_term_department[term][dept] = parsed_classes
        


# write the data from all_classes_by_term_department to a json file
with open("testclasses-ALL-1242-1244-1248.json", "w") as f: # search/fixtures/search/ is future location
    json.dump(all_classes_by_term_department, f)


In [28]:
# open the json file and read the data
with open("testclasses-ALL-1242-1244-1248.json", "r") as f:
    data = json.load(f)

for term in terms:
    for dept in prev_department_short_names_combined:
        if isinstance(data[str(term)][dept], list):
            continue
        else:
            print("Error occurred while parsing the file for department: {} and term: {}".format(dept, term))
            print(data[str(term)][dept]["error"])
            print(data[str(term)][dept]["last_class"])
            print()
            print()

In [5]:
with open("testclasses-ALL-1242-1244-1248.json", "r") as f:
    data = json.load(f)


# check the "type" of each class, using a counter
types = Counter()
for term in terms:
    for dept in prev_department_short_names_combined:
        # check the "type"
        for class_ in data[str(term)][dept]:
            for course in class_:
                types[course["type"]] += 1
                # if course["type"] not in ["L", "B"]:
                #     print("Error: class type is not L or B")
                #     print(course)
                #     print()
                #     print()

print(types)

Counter({'L': 20257, 'B': 8099, 'S': 3644, 'R': 2971, 'F': 2534, 'C': 874, 'I': 694, 'W': 296})


In [7]:
types_autoenroll_not_null = Counter()
for term in terms:
    for dept in prev_department_short_names_combined:
        # check the "type"
        for class_ in data[str(term)][dept]:
            for course in class_:
                if course["autoenrolls"] != "":
                    types_autoenroll_not_null[course["type"]] += 1
print(types_autoenroll_not_null)


Counter({'R': 2308, 'B': 2270, 'C': 225, 'L': 202, 'F': 17, 'S': 4, 'W': 2})


In [21]:
# from mechanize import Browser

url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"
url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"

from urllib.parse import parse_qs

payload = "ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=20&ICAction=%23KEY%0D&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=SSR_CLSRCH_WRK_LAST_NAME%248&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=%2BfK%2BzAP6lnEQHtqXnOKYcVhi3IWkJTA2puX8YzKrvi0%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_LAST_NAME$8=bucci"

# Convert the payload into a dictionary
payload_dict = parse_qs(payload)

# Print the dictionary
for key, value in payload_dict.items():
    print(f"{key}: {value[0]}")

response = requests.post(url, payload_dict)
print(response)

# write the response to a html file to view it
with open("test.html", "w") as f:
    f.write(response.text)


ICAJAX: 1
ICNAVTYPEDROPDOWN: 0
ICType: Panel
ICElementNum: 0
ICStateNum: 20
ICAction: #KEY
ICModelCancel: 0
ICXPos: 0
ICYPos: 0
ResponsetoDiffFrame: -1
TargetFrameName: None
FacetPath: None
ICFocus: SSR_CLSRCH_WRK_LAST_NAME$8
ICSaveWarningFilter: 0
ICChanged: -1
ICSkipPending: 0
ICAutoSave: 0
ICResubmit: 0
ICSID: +fK+zAP6lnEQHtqXnOKYcVhi3IWkJTA2puX8YzKrvi0=
ICActionPrompt: false
ICBcDomData: UnknownValue
SSR_CLSRCH_WRK_LAST_NAME$8: bucci
<Response [200]>


In [20]:
from urllib.parse import parse_qs

payload = "ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=20&ICAction=%23KEY%0D&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=SSR_CLSRCH_WRK_LAST_NAME%248&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=%2BfK%2BzAP6lnEQHtqXnOKYcVhi3IWkJTA2puX8YzKrvi0%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_LAST_NAME$8=bucci"

# Convert the payload into a dictionary
payload_dict = parse_qs(payload)

# Print the dictionary
for key, value in payload_dict.items():
    print(f"{key}: {value[0]}")

ICAJAX: 1
ICNAVTYPEDROPDOWN: 0
ICType: Panel
ICElementNum: 0
ICStateNum: 20
ICAction: #KEY
ICModelCancel: 0
ICXPos: 0
ICYPos: 0
ResponsetoDiffFrame: -1
TargetFrameName: None
FacetPath: None
ICFocus: SSR_CLSRCH_WRK_LAST_NAME$8
ICSaveWarningFilter: 0
ICChanged: -1
ICSkipPending: 0
ICAutoSave: 0
ICResubmit: 0
ICSID: +fK+zAP6lnEQHtqXnOKYcVhi3IWkJTA2puX8YzKrvi0=
ICActionPrompt: false
ICBcDomData: UnknownValue
SSR_CLSRCH_WRK_LAST_NAME$8: bucci


In [2]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from pprint import pprint

In [23]:
def get_all_forms(session, url):
    """Returns all form tags found on a web page's `url` """
    # GET request
    res = session.get(url)
    # for javascript driven website
    # res.html.render()
    soup = BeautifulSoup(res.html.html, "html.parser")
    return soup.find_all("form")

In [24]:
def get_form_details(form):
  """Returns the HTML details of a form,
    including action, method and list of form controls (inputs, etc)"""
  details = {}
  # get the form action (requested URL)
  action = form.attrs.get("action").lower()
  # get the form method (POST, GET, DELETE, etc)
  # if not specified, GET is the default in HTML
  method = form.attrs.get("method", "get").lower()
  # get all form inputs
  inputs = []
  for input_tag in form.find_all("input"):
    # get type of input form control
    input_type = input_tag.attrs.get("type", "text")
    # get name attribute
    input_name = input_tag.attrs.get("name")
    # get the default value of that input tag
    input_value = input_tag.attrs.get("value", "")
    # add everything to that list
    inputs.append({
      "type": input_type,
      "name": input_name,
      "value": input_value
    })

  for select in form.find_all("select"):
    # get the name attribute
    select_name = select.attrs.get("name")
    # set the type as select
    select_type = "select"
    select_options = []
    # the default select value
    select_default_value = ""
    # iterate over options and get the value of each
    for select_option in select.find_all("option"):
      # get the option value used to submit the form
      option_value = select_option.attrs.get("value")
      if option_value:
        select_options.append(option_value)
        if select_option.attrs.get("selected"):
          # if 'selected' attribute is set, set this option as default
          select_default_value = option_value
    if not select_default_value and select_options:
      # if the default is not set, and there are options, take the first option as default
      select_default_value = select_options[0]
    # add the select to the inputs list
    inputs.append({
      "type": select_type,
      "name": select_name,
      "values": select_options,
      "value": select_default_value
    })
  for textarea in form.find_all("textarea"):
    # get the name attribute
    textarea_name = textarea.attrs.get("name")
    # set the type as textarea
    textarea_type = "textarea"
    # get the textarea value
    textarea_value = textarea.attrs.get("value", "")
    # add the textarea to the inputs list
    inputs.append({
      "type": textarea_type,
      "name": textarea_name,
      "value": textarea_value
    })

  details["action"] = action
  details["method"] = method
  details["inputs"] = inputs
  return details

In [26]:
session = HTMLSession()
# get URL from the command line
url = url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"
# get all form tags
forms = get_all_forms(session, url)
# print(forms)
# iteratte over forms
for i, form in enumerate(forms, start=1):
    form_details = get_form_details(form)
    print("=" * 50, f"form #{i}", "=" * 50)
    pprint(form_details)

================================================== form #1 ==================================================
{'action': 'https://courses.erppub.osu.edu/psc/ps/employee/pub/c/community_access.class_search.gbl',
 'inputs': [{'name': 'ICType', 'type': 'hidden', 'value': 'Panel'},
            {'name': 'ICElementNum', 'type': 'hidden', 'value': '0'},
            {'name': 'ICStateNum', 'type': 'hidden', 'value': '1'},
            {'name': 'ICAction', 'type': 'hidden', 'value': 'None'},
            {'name': 'ICModelCancel', 'type': 'hidden', 'value': '0'},
            {'name': 'ICXPos', 'type': 'hidden', 'value': '0'},
            {'name': 'ICYPos', 'type': 'hidden', 'value': '0'},
            {'name': 'ResponsetoDiffFrame', 'type': 'hidden', 'value': '-1'},
            {'name': 'TargetFrameName', 'type': 'hidden', 'value': 'None'},
            {'name': 'FacetPath', 'type': 'hidden', 'value': 'None'},
            {'name': 'ICFocus', 'type': 'hidden', 'value': ''},
            {'name': 'ICSav

In [28]:
session = HTMLSession()

all_forms = get_all_forms(session, url)
# get the first form (edit this as you wish)
# first_form = get_all_forms(url)[0]
for i, f in enumerate(all_forms, start=1):
    form_details = get_form_details(f)
    print(f"{i} #")
    pprint(form_details)
    print("="*50)

choice = int(input("Enter form indice: "))
# extract all form details
form_details = get_form_details(all_forms[choice-1])
pprint(form_details)

# the data body we want to submit
data = {}
for input_tag in form_details["inputs"]:
    if input_tag["type"] == "hidden":
        # if it's hidden, use the default value
        data[input_tag["name"]] = input_tag["value"]
    elif input_tag["type"] == "select":
        for i, option in enumerate(input_tag["values"], start=1):
            # iterate over available select options
            if option == input_tag["value"]:
                print(f"{i} # {option} (default)")
            else:
                print(f"{i} # {option}")
        choice = input(f"Enter the option for the select field '{input_tag['name']}' (1-{i}): ")
        try:
            choice = int(choice)
        except:
            # choice invalid, take the default
            value = input_tag["value"]
        else:
            value = input_tag["values"][choice-1]
        data[input_tag["name"]] = value
    elif input_tag["type"] != "submit":
        # all others except submit, prompt the user to set it
        value = input(f"Enter the value of the field '{input_tag['name']}' (type: {input_tag['type']}): ")
        data[input_tag["name"]] = value

1 #
{'action': 'https://courses.erppub.osu.edu/psc/ps/employee/pub/c/community_access.class_search.gbl',
 'inputs': [{'name': 'ICType', 'type': 'hidden', 'value': 'Panel'},
            {'name': 'ICElementNum', 'type': 'hidden', 'value': '0'},
            {'name': 'ICStateNum', 'type': 'hidden', 'value': '1'},
            {'name': 'ICAction', 'type': 'hidden', 'value': 'None'},
            {'name': 'ICModelCancel', 'type': 'hidden', 'value': '0'},
            {'name': 'ICXPos', 'type': 'hidden', 'value': '0'},
            {'name': 'ICYPos', 'type': 'hidden', 'value': '0'},
            {'name': 'ResponsetoDiffFrame', 'type': 'hidden', 'value': '-1'},
            {'name': 'TargetFrameName', 'type': 'hidden', 'value': 'None'},
            {'name': 'FacetPath', 'type': 'hidden', 'value': 'None'},
            {'name': 'ICFocus', 'type': 'hidden', 'value': ''},
            {'name': 'ICSaveWarningFilter', 'type': 'hidden', 'value': '0'},
            {'name': 'ICChanged', 'type': 'hidden', 'val

In [29]:
from urllib.parse import urljoin
# join the url with the action (form request URL)
url = urljoin(url, form_details["action"])
# pprint(data)
if form_details["method"] == "post":
    res = session.post(url, data=data)
elif form_details["method"] == "get":
    res = session.get(url, params=data)

In [30]:
# the data body we want to submit
data = {}
for input_tag in form_details["inputs"]:
    if input_tag["type"] == "hidden":
        # if it's hidden, use the default value
        data[input_tag["name"]] = input_tag["value"]
    elif input_tag["type"] == "select":
        for i, option in enumerate(input_tag["values"], start=1):
            # iterate over available select options
            if option == input_tag["value"]:
                print(f"{i} # {option} (default)")
            else:
                print(f"{i} # {option}")
        choice = input(f"Enter the option for the select field '{input_tag['name']}' (1-{i}): ")
        try:
            choice = int(choice)
        except:
            # choice invalid, take the default
            value = input_tag["value"]
        else:
            value = input_tag["values"][choice-1]
        data[input_tag["name"]] = value
    elif input_tag["type"] != "submit":
        # all others except submit, prompt the user to set it
        value = input(f"Enter the value of the field '{input_tag['name']}' (type: {input_tag['type']}): ")
        data[input_tag["name"]] = value

1 # OSUSI (default)
1 # 1248
2 # 1242 (default)
3 # 1244
1 # COL (default)
2 # LMA
3 # MNS
4 # MRN
5 # NWK
6 # WST
1 # ACADAFF (default)
2 # ACCTMIS
3 # ACCAD
4 # AEROENG
5 # AFAMAST
6 # AEDECON
7 # ACEL
8 # AGRCOMM
9 # AGSYSMT
10 # ASE
11 # AIRSCI
12 # AMINSTS
13 # ASL
14 # ANATOMY
15 # ANIMSCI
16 # ANMLTEC
17 # ANTHROP
18 # ARABIC
19 # ARCH
20 # ART
21 # ARTEDUC
22 # ARTSSCI
23 # ASAMSTS
24 # ASTRON
25 # ATHTRNG
26 # ATMOSSC
27 # AVIATN
28 # BIOCHEM
29 # BIOETHC
30 # BIOPHRM
31 # BIOLOGY
32 # BIOMEDE
33 # BMI
34 # BIOMSCI
35 # BSGP
36 # BIOPHYS
37 # BIOTECH
38 # BCS
39 # BUSOBA
40 # BUSFIN
41 # BUSMHR
42 # BUSML
43 # BUSADM
44 # BUSTEC
45 # CONSRTM
46 # CBG
47 # CATALAN
48 # CLLC
49 # CHEMPHY
50 # CBE
51 # CHEM
52 # CHINESE
53 # CRPLAN
54 # CIVILEN
55 # CLAS
56 # MCR
57 # COMM
58 # COMLDR
59 # COMPSTD
60 # CSE
61 # CONSYSM
62 # CSCFFS
63 # CSFRST
64 # CSHSPMG
65 # CONSCI
66 # CRPSOIL
67 # CSTW
68 # CZECH
69 # DANCE
70 # DENTHYG
71 # DENT
72 # DESIGN
73 # DSABLST
74 # DNE
75 # EARTHSC

In [31]:
# join the url with the action (form request URL)
url = urljoin(url, form_details["action"])
# pprint(data)
if form_details["method"] == "post":
    res = session.post(url, data=data)
elif form_details["method"] == "get":
    res = session.get(url, params=data)

In [32]:
# the below code is only for replacing relative URLs to absolute ones
soup = BeautifulSoup(res.content, "html.parser")
for link in soup.find_all("link"):
    try:
        link.attrs["href"] = urljoin(url, link.attrs["href"])
    except:
        pass
for script in soup.find_all("script"):
    try:
        script.attrs["src"] = urljoin(url, script.attrs["src"])
    except:
        pass
for img in soup.find_all("img"):
    try:
        img.attrs["src"] = urljoin(url, img.attrs["src"])
    except:
        pass
for a in soup.find_all("a"):
    try:
        a.attrs["href"] = urljoin(url, a.attrs["href"])
    except:
        pass

# write the page content to a file
open("page.html", "w").write(str(soup))

254

In [38]:
cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'CS-PORTAL-PSJSESSIONID': '6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150',
    'PS_TOKEN': 'rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABQixQtzVMZXf7dMFJMHmL83VWfsg24AAAAFAFNkYXRhYnicHYpLDkBAEAXLECewcgdiGJ+tCDaC+Kydxb0czqOTfq+r0jcQ+Mbz1I/hn7hj4WRjp9dejKLvOgh7ZmG0igbxxaTfjlbtcjJyHIm6UNo/P5vKlkpLpWyo5axcDS/Tvw6s',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'PS_TOKENEXPIRE': '14_Mar_2024_04:59:06_GMT',
    'AWSALB': 'FnKcklSDfCHM/WO8REFa9rbWdsRHEcW8IZxuf3bc/ouV/irY9ZGDG1fkam3J/aob4bfIueWpZex1/gi/PLx4VWTc7kBw3QaY+YZM0N6RMFtQDZ9ZWSK6ZA+aAOOS',
    'AWSALBCORS': 'FnKcklSDfCHM/WO8REFa9rbWdsRHEcW8IZxuf3bc/ouV/irY9ZGDG1fkam3J/aob4bfIueWpZex1/gi/PLx4VWTc7kBw3QaY+YZM0N6RMFtQDZ9ZWSK6ZA+aAOOS',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABQixQtzVMZXf7dMFJMHmL83VWfsg24AAAAFAFNkYXRhYnicHYpLDkBAEAXLECewcgdiGJ+tCDaC+Kydxb0czqOTfq+r0jcQ+Mbz1I/hn7hj4WRjp9dejKLvOgh7ZmG0igbxxaTfjlbtcjJyHIm6UNo/P5vKlkpLpWyo5axcDS/Tvw6s; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_04:59:06_GMT; AWSALB=FnKcklSDfCHM/WO8REFa9rbWdsRHEcW8IZxuf3bc/ouV/irY9ZGDG1fkam3J/aob4bfIueWpZex1/gi/PLx4VWTc7kBw3QaY+YZM0N6RMFtQDZ9ZWSK6ZA+aAOOS; AWSALBCORS=FnKcklSDfCHM/WO8REFa9rbWdsRHEcW8IZxuf3bc/ouV/irY9ZGDG1fkam3J/aob4bfIueWpZex1/gi/PLx4VWTc7kBw3QaY+YZM0N6RMFtQDZ9ZWSK6ZA+aAOOS',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=12&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=x%2BLBQqNicONH3QazVUZKgcsrYifYjSDt%2FAYQiZnlSN8%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_LAST_NAME$8=bucci&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE'

# Convert the payload into a dictionary
payload_dict = parse_qs(data)

payload_dict["SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4"] = ["N"]
payload_dict["SSR_CLSRCH_WRK_LAST_NAME$8"] = ["bucci"]

# Print the dictionary
for key, value in payload_dict.items():
    print(f"{key}: {value[0]}")


response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=payload_dict)

print(response)

# write the response to a html file to view it
with open("test.html", "w") as f:
    f.write(response.text)

ICAJAX: 1
ICNAVTYPEDROPDOWN: 0
ICType: Panel
ICElementNum: 0
ICStateNum: 12
ICAction: CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH
ICModelCancel: 0
ICXPos: 0
ICYPos: 0
ResponsetoDiffFrame: -1
TargetFrameName: None
FacetPath: None
ICSaveWarningFilter: 0
ICChanged: -1
ICSkipPending: 0
ICAutoSave: 0
ICResubmit: 0
ICSID: x+LBQqNicONH3QazVUZKgcsrYifYjSDt/AYQiZnlSN8=
ICActionPrompt: false
ICBcDomData: UnknownValue
SSR_CLSRCH_WRK_LAST_NAME$8: bucci
SSR_CLSRCH_WRK_SUBJECT_SRCH$1: CSE
SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4: N
<Response [200]>


In [47]:
import requests

cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'CS-PORTAL-PSJSESSIONID': '6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'PS_TOKENEXPIRE': '14_Mar_2024_05:32:57_GMT',
    'AWSALB': 'OktoiOYC/jP+FE3Gzb0WeUI5NbIS6NOah/3RZtBrdX7Z0Lnm7gu5H8kFXs5Glp33DrFyUQB3Fpi9Jz67LV5Z8NymZBRTQ6Zmjxx6IM1XSz+YYflfT0JSgahhcymF',
    'AWSALBCORS': 'OktoiOYC/jP+FE3Gzb0WeUI5NbIS6NOah/3RZtBrdX7Z0Lnm7gu5H8kFXs5Glp33DrFyUQB3Fpi9Jz67LV5Z8NymZBRTQ6Zmjxx6IM1XSz+YYflfT0JSgahhcymF',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_05:33:02_GMT; AWSALB=VfDBUKxPNOHBXat8pytD2mzsrN08wVLIysP2a9H1E8K/OnYnA5uIFVtoaa8djTVDMWgRCdUYSy5IhLaFOACXLEneSfIQsiTjPClpc/o+AzzAijm2kVZr+97wI0Xo; AWSALBCORS=VfDBUKxPNOHBXat8pytD2mzsrN08wVLIysP2a9H1E8K/OnYnA5uIFVtoaa8djTVDMWgRCdUYSy5IhLaFOACXLEneSfIQsiTjPClpc/o+AzzAijm2kVZr+97wI0Xo',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

# data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=25&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=x%2BLBQqNicONH3QazVUZKgcsrYifYjSDt%2FAYQiZnlSN8%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_LAST_NAME$8=latour&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE'
data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=48&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=x%2BLBQqNicONH3QazVUZKgcsrYifYjSDt%2FAYQiZnlSN8%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_LAST_NAME$8=latour&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE'

# Convert the payload into a dictionary
payload_dict = parse_qs(data)

# payload_dict["SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4"] = ["N"]
payload_dict["SSR_CLSRCH_WRK_LAST_NAME$8"] = ["bucci"]

# Print the dictionary
for key, value in payload_dict.items():
    print(f"{key}: {value[0]}")


response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=payload_dict)

print(response)

# write the response to a html file to view it
with open("test.html", "w") as f:
    f.write(response.text)

ICAJAX: 1
ICNAVTYPEDROPDOWN: 0
ICType: Panel
ICElementNum: 0
ICStateNum: 48
ICAction: CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH
ICModelCancel: 0
ICXPos: 0
ICYPos: 0
ResponsetoDiffFrame: -1
TargetFrameName: None
FacetPath: None
ICSaveWarningFilter: 0
ICChanged: -1
ICSkipPending: 0
ICAutoSave: 0
ICResubmit: 0
ICSID: x+LBQqNicONH3QazVUZKgcsrYifYjSDt/AYQiZnlSN8=
ICActionPrompt: false
ICBcDomData: UnknownValue
SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4: N
SSR_CLSRCH_WRK_LAST_NAME$8: bucci
SSR_CLSRCH_WRK_SUBJECT_SRCH$1: CSE
<Response [200]>


In [91]:
data = "PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"

response = requests.get("https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes", 
                        data=data, 
                        headers={
                            "authority": "courses.erppub.osu.edu",
                            "method": "GET",
                            "path": "/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes",
                            "scheme": "https",
                            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
                            "Accept-encoding": "gzip, deflate, br, zstd",
                            "Accept-language": "en-US,en;q=0.9",
                            "Cache-Control": "no-cache",
                            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
                            "Dnt": "1",
                            "Pragma": "no-cache",
                            "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
                            "Sec-Ch-Ua-Mobile": "?0",
                            "Sec-Fetch-Dest": "document",
                            "Sec-Ch-Ua-Platform": "Windows",
                            "Sec-Fetch-Mode": "navigate",
                            "Sec-Fetch-Site": "none",
                            "Sec-Fetch-User": "?1",
                            "Upgrade-Insecure-Requests": "1",
                        })


temp_cookies = response.request.headers
print(temp_cookies)

print(response.headers)

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', 'Accept-encoding': 'gzip, deflate, br, zstd', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'Connection': 'keep-alive', 'authority': 'courses.erppub.osu.edu', 'method': 'GET', 'path': '/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes', 'scheme': 'https', 'Accept-language': 'en-US,en;q=0.9', 'Cache-Control': 'no-cache', 'Dnt': '1', 'Pragma': 'no-cache', 'Sec-Ch-Ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v

In [83]:
response = requests.get("https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes")

temp_cookies = response.request.headers["Cookie"]
temp_cookies = temp_cookies.split("; ")
new_cookies = {}
for cookie in temp_cookies:
    key, value = cookie.split("=")
    new_cookies[key] = value

cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'CS-PORTAL-PSJSESSIONID': '6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'PS_TOKENEXPIRE': '',
    'AWSALB': '',
    'AWSALBCORS': '',
}

cookies["PS_TOKENEXPIRE"] = new_cookies["PS_TOKENEXPIRE"]
cookies["AWSALB"] = new_cookies["AWSALB"]
cookies["AWSALBCORS"] = new_cookies["AWSALBCORS"]

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_05:33:02_GMT; AWSALB=VfDBUKxPNOHBXat8pytD2mzsrN08wVLIysP2a9H1E8K/OnYnA5uIFVtoaa8djTVDMWgRCdUYSy5IhLaFOACXLEneSfIQsiTjPClpc/o+AzzAijm2kVZr+97wI0Xo; AWSALBCORS=VfDBUKxPNOHBXat8pytD2mzsrN08wVLIysP2a9H1E8K/OnYnA5uIFVtoaa8djTVDMWgRCdUYSy5IhLaFOACXLEneSfIQsiTjPClpc/o+AzzAijm2kVZr+97wI0Xo',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=48&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=x%2BLBQqNicONH3QazVUZKgcsrYifYjSDt%2FAYQiZnlSN8%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_LAST_NAME$8=latour&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE'

# Convert the payload into a dictionary
payload_dict = parse_qs(data)

# payload_dict["SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4"] = ["N"]
payload_dict["SSR_CLSRCH_WRK_LAST_NAME$8"] = ["bucci"]

# Print the dictionary
for key, value in payload_dict.items():
    print(f"{key}: {value[0]}")


response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=payload_dict)

print(response)

# write the response to a html file to view it
with open("test.html", "w") as f:
    f.write(response.text)

KeyError: 'PS_TOKENEXPIRE'

In [59]:
# session = requests.Session() 

# response = session.get("https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes")
# pprint(session.cookies.get_dict())

cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'CS-PORTAL-PSJSESSIONID': '6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'PS_TOKENEXPIRE': '14_Mar_2024_05:33:10_GMT',
    'AWSALB': 'm/xkeItzRTz/cSyw2RTEQRV9Kw6L18tjm4NC1cEfr7EDeUKt8NI0tJVew8OVFLCLxPoAI9xCuO7UDk5BW8RapiAiZ9xXtbYMPEyLyqxhX7a2npq0CVO08KinQFXV',
    'AWSALBCORS': 'm/xkeItzRTz/cSyw2RTEQRV9Kw6L18tjm4NC1cEfr7EDeUKt8NI0tJVew8OVFLCLxPoAI9xCuO7UDk5BW8RapiAiZ9xXtbYMPEyLyqxhX7a2npq0CVO08KinQFXV',
}

# new_cookies = session.cookies.get_dict()

# # replace the keys in cookies with the new keys
# for key in new_cookies:
#     cookies[key] = new_cookies[key]

pprint(cookies)

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_05:33:02_GMT; AWSALB=VfDBUKxPNOHBXat8pytD2mzsrN08wVLIysP2a9H1E8K/OnYnA5uIFVtoaa8djTVDMWgRCdUYSy5IhLaFOACXLEneSfIQsiTjPClpc/o+AzzAijm2kVZr+97wI0Xo; AWSALBCORS=VfDBUKxPNOHBXat8pytD2mzsrN08wVLIysP2a9H1E8K/OnYnA5uIFVtoaa8djTVDMWgRCdUYSy5IhLaFOACXLEneSfIQsiTjPClpc/o+AzzAijm2kVZr+97wI0Xo',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=48&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=x%2BLBQqNicONH3QazVUZKgcsrYifYjSDt%2FAYQiZnlSN8%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_LAST_NAME$8=latour&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE'

# Convert the payload into a dictionary
payload_dict = parse_qs(data)

# payload_dict["SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4"] = ["N"]
payload_dict["SSR_CLSRCH_WRK_LAST_NAME$8"] = ["bucci"]

# Print the dictionary
for key, value in payload_dict.items():
    print(f"{key}: {value[0]}")


response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=payload_dict)

print(response)

# write the response to a html file to view it
with open("test.html", "w") as f:
    f.write(response.text)

# session.close()

{'AWSALB': 'm/xkeItzRTz/cSyw2RTEQRV9Kw6L18tjm4NC1cEfr7EDeUKt8NI0tJVew8OVFLCLxPoAI9xCuO7UDk5BW8RapiAiZ9xXtbYMPEyLyqxhX7a2npq0CVO08KinQFXV',
 'AWSALBCORS': 'm/xkeItzRTz/cSyw2RTEQRV9Kw6L18tjm4NC1cEfr7EDeUKt8NI0tJVew8OVFLCLxPoAI9xCuO7UDk5BW8RapiAiZ9xXtbYMPEyLyqxhX7a2npq0CVO08KinQFXV',
 'CS-PORTAL-PSJSESSIONID': '6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150',
 'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
 'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
 'PS_DEVICEFEATURES': 'new:1',
 'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
 'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
 'PS_TOKENEXPIRE': '14_Mar_2024_05:33:10_GMT',
 'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
 'SignOnDefault': '',
 '_ce.s': 'v~1105b66

In [98]:
import requests

cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'lcsrftoken': 'KExa6dPVE05zl0m+hGJYjp755TeanSlu6nNSx8KrqYE=',
    'IOS_FULLSCREEN': '0',
    'CS-PORTAL-PSJSESSIONID': '1tc7rsv9S3N-5LzSjf6YuFL0r6-g4m8G!697372150',
    'PS_TOKEN': 'rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSGBwIOQRzHs57Hnv/Lx8+9Cv5r8G4AAAAFAFNkYXRhYnicHYpLDkBAEETfIE5g5Q4mDMnYimAjiM/aWdzL4ZTpTlfVS9cDJHFkjPyNCJP3rFzsHAy6m0n0p5N0YBFmm2gU38zq9nTyxlHiaCjktbQK6rRWqZVW+lp86HmRhw/T1g6r',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'PS_TOKENEXPIRE': '14_Mar_2024_06:52:31_GMT',
    'AWSALB': 'K6IkcTu1g9Vys+eag9Q4dDh8B/y1vXfnuESUe8n1PdYdEYac0tLAPa/EiJHlX2aQq3l/J/lFSYfwPf6q87Sf2GjOSDk6zwegTzobWWMNDrpFAZUtjD0Q2zeMfShL',
    'AWSALBCORS': 'K6IkcTu1g9Vys+eag9Q4dDh8B/y1vXfnuESUe8n1PdYdEYac0tLAPa/EiJHlX2aQq3l/J/lFSYfwPf6q87Sf2GjOSDk6zwegTzobWWMNDrpFAZUtjD0Q2zeMfShL',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; lcsrftoken=KExa6dPVE05zl0m+hGJYjp755TeanSlu6nNSx8KrqYE=; IOS_FULLSCREEN=0; CS-PORTAL-PSJSESSIONID=1tc7rsv9S3N-5LzSjf6YuFL0r6-g4m8G!697372150; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSGBwIOQRzHs57Hnv/Lx8+9Cv5r8G4AAAAFAFNkYXRhYnicHYpLDkBAEETfIE5g5Q4mDMnYimAjiM/aWdzL4ZTpTlfVS9cDJHFkjPyNCJP3rFzsHAy6m0n0p5N0YBFmm2gU38zq9nTyxlHiaCjktbQK6rRWqZVW+lp86HmRhw/T1g6r; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; PS_TOKENEXPIRE=14_Mar_2024_06:52:31_GMT; AWSALB=K6IkcTu1g9Vys+eag9Q4dDh8B/y1vXfnuESUe8n1PdYdEYac0tLAPa/EiJHlX2aQq3l/J/lFSYfwPf6q87Sf2GjOSDk6zwegTzobWWMNDrpFAZUtjD0Q2zeMfShL; AWSALBCORS=K6IkcTu1g9Vys+eag9Q4dDh8B/y1vXfnuESUe8n1PdYdEYac0tLAPa/EiJHlX2aQq3l/J/lFSYfwPf6q87Sf2GjOSDk6zwegTzobWWMNDrpFAZUtjD0Q2zeMfShL',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=11&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=TlzXuK8FQ1AJuJJ856FzpSF7LKz7gu%2B7MmrmL%2BFvsLE%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE&SSR_CLSRCH_WRK_SSR_EXACT_MATCH1$2=E&SSR_CLSRCH_WRK_CATALOG_NBR$2=&SSR_CLSRCH_WRK_ACAD_CAREER$3=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_OEE_IND$chk$5=N&SSR_CLSRCH_WRK_SSR_START_TIME_OPR$6=GE&SSR_CLSRCH_WRK_MEETING_TIME_START$6=&SSR_CLSRCH_WRK_SSR_END_TIME_OPR$6=LE&SSR_CLSRCH_WRK_MEETING_TIME_END$6=&SSR_CLSRCH_WRK_INCLUDE_CLASS_DAYS$7=I&SSR_CLSRCH_WRK_MON$chk$7=&SSR_CLSRCH_WRK_TUES$chk$7=&SSR_CLSRCH_WRK_WED$chk$7=&SSR_CLSRCH_WRK_THURS$chk$7=&SSR_CLSRCH_WRK_LAST_NAME$8=bucci'

response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=data)

with open("test.html", "w") as f:
    f.write(response.text)
    

In [107]:
session = requests.Session()
r = session.get('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL')
# print(r.request.headers)
print(r.cookies.get_dict())

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; lcsrftoken=KExa6dPVE05zl0m+hGJYjp755TeanSlu6nNSx8KrqYE=; IOS_FULLSCREEN=0; CS-PORTAL-PSJSESSIONID=1tc7rsv9S3N-5LzSjf6YuFL0r6-g4m8G!697372150; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSGBwIOQRzHs57Hnv/Lx8+9Cv5r8G4AAAAFAFNkYXRhYnicHYpLDkBAEETfIE5g5Q4mDMnYimAjiM/aWdzL4ZTpTlfVS9cDJHFkjPyNCJP3rFzsHAy6m0n0p5N0YBFmm2gU38zq9nTyxlHiaCjktbQK6rRWqZVW+lp86HmRhw/T1g6r; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; PS_TOKENEXPIRE=14_Mar_2024_06:52:31_GMT; AWSALB=K6IkcTu1g9Vys+eag9Q4dDh8B/y1vXfnuESUe8n1PdYdEYac0tLAPa/EiJHlX2aQq3l/J/lFSYfwPf6q87Sf2GjOSDk6zwegTzobWWMNDrpFAZUtjD0Q2zeMfShL; AWSALBCORS=K6IkcTu1g9Vys+eag9Q4dDh8B/y1vXfnuESUe8n1PdYdEYac0tLAPa/EiJHlX2aQq3l/J/lFSYfwPf6q87Sf2GjOSDk6zwegTzobWWMNDrpFAZUtjD0Q2zeMfShL',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=11&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=TlzXuK8FQ1AJuJJ856FzpSF7LKz7gu%2B7MmrmL%2BFvsLE%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE&SSR_CLSRCH_WRK_SSR_EXACT_MATCH1$2=E&SSR_CLSRCH_WRK_CATALOG_NBR$2=2231&SSR_CLSRCH_WRK_ACAD_CAREER$3=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_OEE_IND$chk$5=N&SSR_CLSRCH_WRK_SSR_START_TIME_OPR$6=GE&SSR_CLSRCH_WRK_MEETING_TIME_START$6=&SSR_CLSRCH_WRK_SSR_END_TIME_OPR$6=LE&SSR_CLSRCH_WRK_MEETING_TIME_END$6=&SSR_CLSRCH_WRK_INCLUDE_CLASS_DAYS$7=I&SSR_CLSRCH_WRK_MON$chk$7=&SSR_CLSRCH_WRK_TUES$chk$7=&SSR_CLSRCH_WRK_WED$chk$7=&SSR_CLSRCH_WRK_THURS$chk$7='

r = session.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes', cookies=r.cookies.get_dict(), headers=headers, data=data)

print(r)

with open("test.html", "w") as f:
    f.write(r.text)

{'CS-PORTAL-PSJSESSIONID': '7xQ71EYCZr_HqJsj_WNpyKE8Ku5Tq22b!697372150', 'ExpirePage': 'https://courses.erppub.osu.edu/psc/ps/', 'PS_DEVICEFEATURES': 'new:1', 'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/', 'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps', 'PS_TOKEN': 'rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABT2J9TAVNjInrpuAEEBYijR+KFPB20AAAAFAFNkYXRhYXicHYpLDkBAFATLJ05g5Q5kZkjEUibYCOKzdhb3cjhtXtLVXcl7gDSJo0j9xoQrPCsXOweDcjPJ/nWSDSzSfJON8ptZv55e3TgMjoZSXYs20ImVdhdoRaPdhjj4ANP3DrI=', 'PS_TOKENEXPIRE': '14_Mar_2024_07:19:11_GMT', 'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID', 'SignOnDefault': '', 'AWSALB': 'YNoC1S4UUQpi8MIenTtZfDSICzHJ0vVYote/KglWnX466TlWVhm+TAEK+Bt/Z7mkphrwCEEndBYfbnk8+AsKfhIE/E7rs+SrzyuJyZvhoM73CoY4GNc/XBhs165a', 'AWSALBCORS': 'YNoC1S4UUQpi8MIenTtZfDSICzHJ0vVYote/KglWnX466TlWVhm+TAEK+Bt/Z7mkphrwCEEndBYfbnk8+AsKfhIE/E7rs+SrzyuJyZvhoM73CoY4GNc/XBhs165a'}
<Response [200]>


In [110]:
import requests
url = 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL'
headers = {
    'accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',
    'Accept-Language': 'en-US,en;q=0.9,it;q=0.8,es;q=0.7',
    'referer': 'https://www.google.com/',
    'cookie': 'DSID=AAO-7r4OSkS76zbHUkiOpnI0kk-X19BLDFF53G8gbnd21VZV2iehu-w_2v14cxvRvrkd_NjIdBWX7wUiQ66f-D8kOkTKD1BhLVlqrFAaqDP3LodRK2I0NfrObmhV9HsedGE7-mQeJpwJifSxdchqf524IMh9piBflGqP0Lg0_xjGmLKEQ0F4Na6THgC06VhtUG5infEdqMQ9otlJENe3PmOQTC_UeTH5DnENYwWC8KXs-M4fWmDADmG414V0_X0TfjrYu01nDH2Dcf3TIOFbRDb993g8nOCswLMi92LwjoqhYnFdf1jzgK0'
}
response = requests.get(url, headers=headers)
print(response.request.headers)
print(response.text)

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53', 'Accept-Encoding': 'gzip, deflate', 'accept': '*/*', 'Connection': 'keep-alive', 'Accept-Language': 'en-US,en;q=0.9,it;q=0.8,es;q=0.7', 'referer': 'https://www.google.com/', 'Cookie': 'AWSALB=+Zb+KKrBj4jdyG6Xi2rWoQG2yAjPZlW3C2ow/VXdSP4qdn7yeJauq2l7B4VbK5bFiCHyD1gG/A5ktz/v9iKuDaxxXBwJ3YQ2HBu1s8Myh01c+kbLXgyYFaK2ILBS; AWSALBCORS=+Zb+KKrBj4jdyG6Xi2rWoQG2yAjPZlW3C2ow/VXdSP4qdn7yeJauq2l7B4VbK5bFiCHyD1gG/A5ktz/v9iKuDaxxXBwJ3YQ2HBu1s8Myh01c+kbLXgyYFaK2ILBS; CS-PORTAL-PSJSESSIONID=c8s73aVzDKjK3whBnrP6rk23qvBmqubz!790274619'}
<!DOCTYPE html>
<html class='pc chrome edgec mac psc_dir-ltr psc_form-xlarge' dir='ltr' lang='en'>
<!-- Copyright (c) 2000, 2021, Oracle and/or its affiliates.  -->
<!-- CHROME/101.0/MAC; ToolsRel=8.59.21; User=COURSES_GUEST; DB=CSOSU/ORACLE; AppServ=//localhost:9033; -->
<head>
<meta name="apple-mobile-web-app-capabl

In [58]:
session = requests.Session()

try:
    url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"
    data = "PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"
    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        # Requests sorts cookies= alphabetically
        # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; IOS_FULLSCREEN=0; lcsrftoken=g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=; CS-PORTAL-PSJSESSIONID=3eI742RlZZMy8mVPznVlqMr82ZGHkbEN!697372150; PS_LASTSITE=https://courses.erppub.osu.edu/psp/ps/; https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2femployee%2fcamp%2frefresh=list: %3ftab%3dos_facstaff_erp|%3frp%3dos_facstaff_erp|%3ftab%3dremoteunifieddashboard|%3frp%3dremoteunifieddashboard; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_TokenSite=https://courses.erppub.osu.edu/psp/ps/?CS-PORTAL-PSJSESSIONID; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_TOKEN=sAAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABRhoT7WwxaPKi5P6Cv6DYklT4YvsHAAAAAFAFNkYXRhZHicHYrBDYJQEAUHJFbgiR4g8P2EcCQEuRghimdqoS+Lc2CTN29nszuQXdIksX8p5+QDM1/efBjNxqQd28p15KXeFu2hbzz9HejtGKgIRAr7LuuTQZamkdFbSadVbq3NH9PFDqg=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TOKENEXPIRE=14_Mar_2024_07:36:49_GMT; AWSALB=O+o84eEtYo5jNBfVheLvoGvsR8ll3iuryzHbB4PQPXtDuFQDEsFli4ZCyv/rzuiXTxAJZVb3KJ9Ci4M4VTbTzcSxXxktHL2d2x+rh+dTcmu8iA6oM9D1NRF5iEJz; AWSALBCORS=O+o84eEtYo5jNBfVheLvoGvsR8ll3iuryzHbB4PQPXtDuFQDEsFli4ZCyv/rzuiXTxAJZVb3KJ9Ci4M4VTbTzcSxXxktHL2d2x+rh+dTcmu8iA6oM9D1NRF5iEJz; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
        'dnt': '1',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    cookies = {
        '_tt_enable_cookie': '1',
        '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
        'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
        'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
        '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
        'IOS_FULLSCREEN': '0',
        'lcsrftoken': 'g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=',
        'CS-PORTAL-PSJSESSIONID': '3eI742RlZZMy8mVPznVlqMr82ZGHkbEN!697372150',
        'PS_LASTSITE': 'https://courses.erppub.osu.edu/psp/ps/',
        'https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2femployee%2fcamp%2frefresh': 'list: %3ftab%3dos_facstaff_erp|%3frp%3dos_facstaff_erp|%3ftab%3dremoteunifieddashboard|%3frp%3dremoteunifieddashboard',
        'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
        'PS_TokenSite': 'https://courses.erppub.osu.edu/psp/ps/?CS-PORTAL-PSJSESSIONID',
        'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
        'SignOnDefault': '',
        'PS_DEVICEFEATURES': 'new:1',
        'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
        'PS_TOKENEXPIRE': '',
        'AWSALB': '',
        'AWSALBCORS': '',
        'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    }
    response = session.get(url, headers=headers, data=data, cookies=cookies)

    with open("test1.html", "w") as f:
        f.write(response.text)

    new_cookies = session.cookies.get_dict()
    pprint(new_cookies)


    cookies = {
        '_tt_enable_cookie': '1',
        '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
        'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
        'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
        '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
        'IOS_FULLSCREEN': '0',
        'lcsrftoken': 'g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=',
        'CS-PORTAL-PSJSESSIONID': new_cookies['CS-PORTAL-PSJSESSIONID'],
        'PS_LASTSITE': 'https://courses.erppub.osu.edu/psp/ps/',
        'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
        'PS_TokenSite': 'https://courses.erppub.osu.edu/psp/ps/?CS-PORTAL-PSJSESSIONID',
        'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
        'SignOnDefault': '',
        'PS_TOKEN': new_cookies['PS_TOKEN'],
        'PS_DEVICEFEATURES': 'new:1',
        'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
        'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
        'PS_TOKENEXPIRE': new_cookies['PS_TOKENEXPIRE'],
        'AWSALB': new_cookies['AWSALB'],
        'AWSALBCORS': new_cookies['AWSALBCORS'],
    }

    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        'content-type': 'application/x-www-form-urlencoded',
        # Requests sorts cookies= alphabetically
        # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; IOS_FULLSCREEN=0; lcsrftoken=g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=; CS-PORTAL-PSJSESSIONID=3eI742RlZZMy8mVPznVlqMr82ZGHkbEN!697372150; PS_LASTSITE=https://courses.erppub.osu.edu/psp/ps/; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_TokenSite=https://courses.erppub.osu.edu/psp/ps/?CS-PORTAL-PSJSESSIONID; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_TOKEN=sAAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABRhoT7WwxaPKi5P6Cv6DYklT4YvsHAAAAAFAFNkYXRhZHicHYrBDYJQEAUHJFbgiR4g8P2EcCQEuRghimdqoS+Lc2CTN29nszuQXdIksX8p5+QDM1/efBjNxqQd28p15KXeFu2hbzz9HejtGKgIRAr7LuuTQZamkdFbSadVbq3NH9PFDqg=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; PS_TOKENEXPIRE=14_Mar_2024_08:04:30_GMT; AWSALB=voE53hEUkQ9bNSocaSVECtLR3jNk6JS86OGE74rSzXzUmeYhXWi/l0gzIBa4tVtIIA8R2BRsH7RknrsxurfUnRZOIDfW6rVlcYhseDJgwO9JTIl/aykLSqsn5I50; AWSALBCORS=voE53hEUkQ9bNSocaSVECtLR3jNk6JS86OGE74rSzXzUmeYhXWi/l0gzIBa4tVtIIA8R2BRsH7RknrsxurfUnRZOIDfW6rVlcYhseDJgwO9JTIl/aykLSqsn5I50',
        'dnt': '1',
        'origin': 'https://courses.erppub.osu.edu',
        'pragma': 'no-cache',
        'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=13&ICAction=CLASS_SRCH_WRK2_STRM%2435%24&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=n63qqpMeSzTzHzZMA%2B5%2B2in%2FtdnwYSbtPFxtwy8owdg%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_INSTITUTION$31$=OSUSI&CLASS_SRCH_WRK2_STRM$35$=1248'

    response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=data)

    with open("test2.html", "w") as f:
        f.write(response.text)


    # set cookies for the session
    # new_cookies = response.cookies.get_dict()

    cookies["PS_TOKEN"] = new_cookies["PS_TOKEN"]
    cookies["CS-PORTAL-PSJSESSIONID"] = new_cookies["CS-PORTAL-PSJSESSIONID"]
    cookies["AWSALB"] = new_cookies["AWSALB"]
    cookies["AWSALBCORS"] = new_cookies["AWSALBCORS"]
    cookies["PS_TOKENEXPIRE"] = new_cookies["PS_TOKENEXPIRE"]
    # convert response.request.headers to a dictionary
    pprint(session.cookies.get_dict())

    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        'content-type': 'application/x-www-form-urlencoded',
        # Requests sorts cookies= alphabetically
        # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=6Z87R1-TS2HAOG9I4Isz47KRcGwQppnl!697372150; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABQixQtzVMZXf7dMFJMHmL83VWfsg24AAAAFAFNkYXRhYnicHYpLDkBAEAXLECewcgdiGJ+tCDaC+Kydxb0czqOTfq+r0jcQ+Mbz1I/hn7hj4WRjp9dejKLvOgh7ZmG0igbxxaTfjlbtcjJyHIm6UNo/P5vKlkpLpWyo5axcDS/Tvw6s; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_04:59:06_GMT; AWSALB=FnKcklSDfCHM/WO8REFa9rbWdsRHEcW8IZxuf3bc/ouV/irY9ZGDG1fkam3J/aob4bfIueWpZex1/gi/PLx4VWTc7kBw3QaY+YZM0N6RMFtQDZ9ZWSK6ZA+aAOOS; AWSALBCORS=FnKcklSDfCHM/WO8REFa9rbWdsRHEcW8IZxuf3bc/ouV/irY9ZGDG1fkam3J/aob4bfIueWpZex1/gi/PLx4VWTc7kBw3QaY+YZM0N6RMFtQDZ9ZWSK6ZA+aAOOS',
        'dnt': '1',
        'origin': 'https://courses.erppub.osu.edu',
        'pragma': 'no-cache',
        'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=12&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=x%2BLBQqNicONH3QazVUZKgcsrYifYjSDt%2FAYQiZnlSN8%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_LAST_NAME$8=bucci&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE'
    data = 'PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=TlzXuK8FQ1AJuJJ856FzpSF7LKz7gu%2B7MmrmL%2BFvsLE%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE&SSR_CLSRCH_WRK_SSR_EXACT_MATCH1$2=E&SSR_CLSRCH_WRK_CATALOG_NBR$2=2231&SSR_CLSRCH_WRK_ACAD_CAREER$3=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_OEE_IND$chk$5=N&SSR_CLSRCH_WRK_SSR_START_TIME_OPR$6=GE&SSR_CLSRCH_WRK_MEETING_TIME_START$6=&SSR_CLSRCH_WRK_SSR_END_TIME_OPR$6=LE&SSR_CLSRCH_WRK_MEETING_TIME_END$6=&SSR_CLSRCH_WRK_INCLUDE_CLASS_DAYS$7=I&SSR_CLSRCH_WRK_MON$chk$7=&SSR_CLSRCH_WRK_TUES$chk$7=&SSR_CLSRCH_WRK_WED$chk$7=&SSR_CLSRCH_WRK_THURS$chk$7='

    response = session.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', data=data)
    print(response)
    with open("test3.html", "w") as f:
        f.write(response.text)
finally:
    session.close()

{'AWSALB': 'qX98HgmB87KInaSjOsCyYCZ3AK7grpV5e4SARhiG6eqlp6G5fSYTZjIDMkXZrm3H8/KUY8fa20JV8UMWtBK4BOyaO722qmCrw8XHjac92oVQO3hpTnecAml80VT/',
 'AWSALBCORS': 'qX98HgmB87KInaSjOsCyYCZ3AK7grpV5e4SARhiG6eqlp6G5fSYTZjIDMkXZrm3H8/KUY8fa20JV8UMWtBK4BOyaO722qmCrw8XHjac92oVQO3hpTnecAml80VT/',
 'CS-PORTAL-PSJSESSIONID': 'b9c8R1YIdejXjaUNkeEeCoElmz78RRih!697372150',
 'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
 'PS_TOKEN': 'rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABTcP5YrEpqvjU+Jfc/s3wVgMd19nm0AAAAFAFNkYXRhYXicLYpLDkBAEAXLJ05g5Q6E8VvLBBtBfNbO4l4O55nopLtevfQNhIHveeLj4yaxLJxs7PTai1H2pYOoZ5bGq2yQX0z6tXRiZcgxVKRiqVv8uSBzfUbtciPmtOoMvNNQDqM=',
 'PS_TOKENEXPIRE': '14_Mar_2024_09:24:51_GMT',
 'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
 'SignOnDefault': ''}
{'AWSALB': 'qX98HgmB87KInaSjOsCyYCZ3AK7grpV5e4SARhiG6eqlp6G5fSYTZjIDMkXZrm3H8/KUY8fa20JV8UMWtBK4BOyaO722qmCrw8XHjac92oVQO3hpTnecAml80VT/',
 'AWSALBCORS': 'qX98HgmB87KInaSjOsCyYCZ3AK7grpV5

In [62]:
import requests

cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'IOS_FULLSCREEN': '0',
    'lcsrftoken': 'g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'CS-PORTAL-PSJSESSIONID': '0Cc8MGo-fTgwwBKCn5JO5c_wzP9gaP37!697372150',
    'PS_TOKEN': 'rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSCzo27+ZQhe0ws5x4LWWTaqDIAlW0AAAAFAFNkYXRhYXicLYpLDkBAEETfIE5g5Q5kMCSWItgI4rN2FvdyOEV0Up+XrgsIfM8Y5e3xXdwyc7Cy0Ukng+htO2HHJIwWUS8+GbVtaZQux5LjSJSFPPu7JaWklrvfK/1rbSt4ANSMDrk=',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'PS_TOKENEXPIRE': '14_Mar_2024_08:59:59_GMT',
    'AWSALB': 't/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW',
    'AWSALBCORS': 't/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; IOS_FULLSCREEN=0; lcsrftoken=g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=0Cc8MGo-fTgwwBKCn5JO5c_wzP9gaP37!697372150; PS_TOKEN=rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSCzo27+ZQhe0ws5x4LWWTaqDIAlW0AAAAFAFNkYXRhYXicLYpLDkBAEETfIE5g5Q5kMCSWItgI4rN2FvdyOEV0Up+XrgsIfM8Y5e3xXdwyc7Cy0Ukng+htO2HHJIwWUS8+GbVtaZQux5LjSJSFPPu7JaWklrvfK/1rbSt4ANSMDrk=; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_08:59:59_GMT; AWSALB=t/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW; AWSALBCORS=t/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=3&ICAction=CLASS_SRCH_WRK2_STRM%2435%24&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lTqFb%2FiC91gYFoukXcxfiop0v5ji7qtEB3Q8n4Goizs%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_INSTITUTION$31$=OSUSI&CLASS_SRCH_WRK2_STRM$35$=1248'

response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=data)

with open("test.html", "w") as f:
    f.write(response.text)

In [65]:
cookies = {
    '_tt_enable_cookie': '1',
    '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
    '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
    'IOS_FULLSCREEN': '0',
    'lcsrftoken': 'g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'new:1',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'CS-PORTAL-PSJSESSIONID': '0Cc8MGo-fTgwwBKCn5JO5c_wzP9gaP37!697372150',
    'PS_TOKEN': 'rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSCzo27+ZQhe0ws5x4LWWTaqDIAlW0AAAAFAFNkYXRhYXicLYpLDkBAEETfIE5g5Q5kMCSWItgI4rN2FvdyOEV0Up+XrgsIfM8Y5e3xXdwyc7Cy0Ukng+htO2HHJIwWUS8+GbVtaZQux5LjSJSFPPu7JaWklrvfK/1rbSt4ANSMDrk=',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
    'PS_TOKENEXPIRE': '14_Mar_2024_08:59:59_GMT',
    'AWSALB': 't/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW',
    'AWSALBCORS': 't/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; IOS_FULLSCREEN=0; lcsrftoken=g6UUFZXRAEYq37kHymfxgubkgPWWYGbHuIpD66o8bqk=; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; SignOnDefault=; PS_DEVICEFEATURES=new:1; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalActualURL%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252fEMPLOYEE%252fPUB%252fc%252fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%26PortalRegistryName%3DEMPLOYEE%26PortalServletURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsp%252fps%252f%26PortalURI%3Dhttps%253a%252f%252fcourses.erppub.osu.edu%252fpsc%252fps%252f%26PortalHostNode%3DCAMP%26NoCrumbs%3Dyes%26PortalKeyStruct%3Dyes%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; CS-PORTAL-PSJSESSIONID=0Cc8MGo-fTgwwBKCn5JO5c_wzP9gaP37!697372150; PS_TOKEN=rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSCzo27+ZQhe0ws5x4LWWTaqDIAlW0AAAAFAFNkYXRhYXicLYpLDkBAEETfIE5g5Q5kMCSWItgI4rN2FvdyOEV0Up+XrgsIfM8Y5e3xXdwyc7Cy0Ukng+htO2HHJIwWUS8+GbVtaZQux5LjSJSFPPu7JaWklrvfK/1rbSt4ANSMDrk=; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_TOKENEXPIRE=14_Mar_2024_08:59:59_GMT; AWSALB=t/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW; AWSALBCORS=t/gUq3MTwrg+P0oyHAnnna5kGd065enj8acA8geCnBC4czluz5SDttdal6sY693Bu1H+s3qMQhELO3zq/ru7kGkqBYZfD0BpS4qu/lPzfifjcRPzfgtqqnSMZqlW',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=3&ICAction=CLASS_SRCH_WRK2_STRM%2435%24&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lTqFb%2FiC91gYFoukXcxfiop0v5ji7qtEB3Q8n4Goizs%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_INSTITUTION$31$=OSUSI&CLASS_SRCH_WRK2_STRM$35$=1248'

response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=data)

with open("test2.html", "w") as f:
    f.write(response.text)

In [70]:
session = requests.Session()

try:
    full_url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"
    url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL"
    data = "PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"
    cookies = {
        '_tt_enable_cookie': '1',
        '_ttp': 'IubVju5-P0rG-ACi-3O-sfmCaKa',
        'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
        'OptanonConsent': 'isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0',
        '_ce.s': 'v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733',
        'IOS_FULLSCREEN': '0',
        'SignOnDefault': '',
        'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
        'lcsrftoken': 'cbGOst93ZH6aqi0Uc+K+W8Nf+Aj1gbDtQD3eto3uzBs=',
        'ExpirePage': 'https://courses.erppub.osu.edu/psc/ps/',
        'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
        'PS_LASTSITE': 'https://courses.erppub.osu.edu/psc/ps/',
        'CS-PORTAL-PSJSESSIONID': 'BCw-1sT3_0z1cP9yIQ4aME4J8lKwTidf!790274619',
        'PS_TOKEN': 'rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABShqcKo9rpOetgUgPA9vlpNsZ0YeW4AAAAFAFNkYXRhYnicHYpJDkBAFESfIU5g5Q462hC2ItgIYlj3WdzL4ZT+SQ0vvx4gjsIgUL4h/rKBjZuDk1FyzKK/XSQjqzDdRZPYsWg70CvrkoKSmlxZya3vVt3w/4y4kTciK3W08AHTYw6o',
        'AWSALB': '8/77Qb2nXWaipxbOUUOuCKEdXFvNJ486pKjFa5EAISmmww7/5jV06+TiX4lXLC3hqgHcvI7um6C6hxUIq2jZm1OBibhD/11r5F0wVeMroijyKOAL5pMFnVdcKJp1',
        'AWSALBCORS': '8/77Qb2nXWaipxbOUUOuCKEdXFvNJ486pKjFa5EAISmmww7/5jV06+TiX4lXLC3hqgHcvI7um6C6hxUIq2jZm1OBibhD/11r5F0wVeMroijyKOAL5pMFnVdcKJp1',
        'PS_TOKENEXPIRE': '14_Mar_2024_21:31:23_GMT',
        'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    }

    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        # Requests sorts cookies= alphabetically
        # 'cookie': '_tt_enable_cookie=1; _ttp=IubVju5-P0rG-ACi-3O-sfmCaKa; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&AwaitingReconsent=false&isGpcEnabled=0&browserGpcFlag=0; _ce.s=v~1105b6613692dc785667761e233afed4ccf69126~vpv~13~v11.rlc~1688094472566~lcw~1704145836774~v11.fhb~1694707156256~v11.lhb~1694707156256~gtrk.la~lmjcwnot~lva~1703649968062~v11.cs~206184~v11.s~224dacb0-a8ef-11ee-bffb-0bb741051815~v11.sla~1704146149733~lcw~1704146149733; IOS_FULLSCREEN=0; SignOnDefault=; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; lcsrftoken=cbGOst93ZH6aqi0Uc+K+W8Nf+Aj1gbDtQD3eto3uzBs=; ExpirePage=https://courses.erppub.osu.edu/psc/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; CS-PORTAL-PSJSESSIONID=BCw-1sT3_0z1cP9yIQ4aME4J8lKwTidf!790274619; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABShqcKo9rpOetgUgPA9vlpNsZ0YeW4AAAAFAFNkYXRhYnicHYpJDkBAFESfIU5g5Q462hC2ItgIYlj3WdzL4ZT+SQ0vvx4gjsIgUL4h/rKBjZuDk1FyzKK/XSQjqzDdRZPYsWg70CvrkoKSmlxZya3vVt3w/4y4kTciK3W08AHTYw6o; AWSALB=8/77Qb2nXWaipxbOUUOuCKEdXFvNJ486pKjFa5EAISmmww7/5jV06+TiX4lXLC3hqgHcvI7um6C6hxUIq2jZm1OBibhD/11r5F0wVeMroijyKOAL5pMFnVdcKJp1; AWSALBCORS=8/77Qb2nXWaipxbOUUOuCKEdXFvNJ486pKjFa5EAISmmww7/5jV06+TiX4lXLC3hqgHcvI7um6C6hxUIq2jZm1OBibhD/11r5F0wVeMroijyKOAL5pMFnVdcKJp1; PS_TOKENEXPIRE=14_Mar_2024_21:31:23_GMT; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%22%20%22label%22%3A%22Class%20Search%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%220%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
        'dnt': '1',
        'pragma': 'no-cache',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    response = requests.get('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers)
    print(response)
    print(response.request.headers)
    with open("test1.html", "w") as f:
        f.write(response.text)

    new_cookies = session.cookies.get_dict()
    pprint(new_cookies)

    pprint(session.headers)

    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=1&ICAction=CLASS_SRCH_WRK2_STRM%2435%24&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lhyvjj8T0iMnT5mxyqMibS3CTWJSStjbfJ0QaLVKakU%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_INSTITUTION$31$=OSUSI&CLASS_SRCH_WRK2_STRM$35$=1248'

    response = session.post(url, headers=headers, data=data)
    print(response)
    with open("test2.html", "w") as f:
        f.write(response.text)


    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=2&ICAction=DERIVED_CLSRCH_SSR_EXPAND_COLLAPS%24149%24%242&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lhyvjj8T0iMnT5mxyqMibS3CTWJSStjbfJ0QaLVKakU%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_STRM$35$=1248'

    response = session.post(url, headers=headers, data=data)
    print(response)
    with open("test3.html", "w") as f:
        f.write(response.text)

    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=0&ICType=Panel&ICElementNum=0&ICStateNum=3&ICAction=CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lhyvjj8T0iMnT5mxyqMibS3CTWJSStjbfJ0QaLVKakU%3D&ICActionPrompt=false&ICBcDomData=UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4=N&SSR_CLSRCH_WRK_SUBJECT_SRCH$1=CSE&SSR_CLSRCH_WRK_LAST_NAME$8=bucci'
    response = session.post(url, headers=headers, data=data)
    print(response)
    with open("test4.html", "w") as f:
        f.write(response.text)
finally:
    session.close()

<Response [200]>
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'Connection': 'keep-alive', 'authority': 'courses.erppub.osu.edu', 'accept-language': 'en-US,en;q=0.9', 'cache-control': 'no-cache', 'dnt': '1', 'pragma': 'no-cache', 'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"Windows"', 'sec-fetch-dest': 'document', 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'none', 'sec-fetch-user': '?1', 'upgrade-insecure-requests': '1', 'Cookie': 'ExpirePage=https://courses.erppub.osu.edu/psp/ps/; IOS_FULLSCREEN=0; OptanonConsent=isIABGlobal=false&datestamp=Thu+Dec+28+2023+16%3A21%3A01+GMT-0500+(Eastern+Standard+Time)&version=202312.1.0&hosts=&lan

In [90]:
session = requests.Session()
try:

    cookies = {
        'AWSALBTG': 'NMmy/7cPcZF4C5bfd4vFxCpkjfAEAlrvcwjN3itCBMag68QNkmAxU88RkE5oWXWiM7nuLB/P/OQz4eb/DMwlp/7e7sDKGpEaSsSMPbbeqCdizVCZfSOsdR4dKJl2B+9CicI1ehhQzaBn0Ux91QkZPzmTrvWxcT1yD6xBWaVzuUE6CS2QesM=',
        'AWSALBTGCORS': 'NMmy/7cPcZF4C5bfd4vFxCpkjfAEAlrvcwjN3itCBMag68QNkmAxU88RkE5oWXWiM7nuLB/P/OQz4eb/DMwlp/7e7sDKGpEaSsSMPbbeqCdizVCZfSOsdR4dKJl2B+9CicI1ehhQzaBn0Ux91QkZPzmTrvWxcT1yD6xBWaVzuUE6CS2QesM=',
        'AWSALB': 'QEhnWZazMReavGD5fhReZgRFKpJmspiCqASSoNHdRZ1VytpYiix6m7VDR4kQ+LerVz2w5nT2i8c2w7uMasyfeMYtiOv2XriiTUlDCbj5S6DcboWu28zQHfV3j5bfGODPYKi8LAsm+G0jVom+eHR+K1XRXp4OHB9eclZ4h2cJj5QtosvDt2UetHB8wFz2xifE61iVpjMA+v0ivchJ7WltitGpk8EVKo6wy8t+0Q0NcSMJ4QxaLIV5nQV1Lk7xEDX+lK+HB3+RsuMZxDmjNQ==',
        'AWSALBCORS': 'QEhnWZazMReavGD5fhReZgRFKpJmspiCqASSoNHdRZ1VytpYiix6m7VDR4kQ+LerVz2w5nT2i8c2w7uMasyfeMYtiOv2XriiTUlDCbj5S6DcboWu28zQHfV3j5bfGODPYKi8LAsm+G0jVom+eHR+K1XRXp4OHB9eclZ4h2cJj5QtosvDt2UetHB8wFz2xifE61iVpjMA+v0ivchJ7WltitGpk8EVKo6wy8t+0Q0NcSMJ4QxaLIV5nQV1Lk7xEDX+lK+HB3+RsuMZxDmjNQ==',
        'INGRESSAFFINITY': '2265ae43ed32158b',
    }

    headers = {
        'authority': 'courses.osu.edu',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'AWSALBTG=NMmy/7cPcZF4C5bfd4vFxCpkjfAEAlrvcwjN3itCBMag68QNkmAxU88RkE5oWXWiM7nuLB/P/OQz4eb/DMwlp/7e7sDKGpEaSsSMPbbeqCdizVCZfSOsdR4dKJl2B+9CicI1ehhQzaBn0Ux91QkZPzmTrvWxcT1yD6xBWaVzuUE6CS2QesM=; AWSALBTGCORS=NMmy/7cPcZF4C5bfd4vFxCpkjfAEAlrvcwjN3itCBMag68QNkmAxU88RkE5oWXWiM7nuLB/P/OQz4eb/DMwlp/7e7sDKGpEaSsSMPbbeqCdizVCZfSOsdR4dKJl2B+9CicI1ehhQzaBn0Ux91QkZPzmTrvWxcT1yD6xBWaVzuUE6CS2QesM=; AWSALB=QEhnWZazMReavGD5fhReZgRFKpJmspiCqASSoNHdRZ1VytpYiix6m7VDR4kQ+LerVz2w5nT2i8c2w7uMasyfeMYtiOv2XriiTUlDCbj5S6DcboWu28zQHfV3j5bfGODPYKi8LAsm+G0jVom+eHR+K1XRXp4OHB9eclZ4h2cJj5QtosvDt2UetHB8wFz2xifE61iVpjMA+v0ivchJ7WltitGpk8EVKo6wy8t+0Q0NcSMJ4QxaLIV5nQV1Lk7xEDX+lK+HB3+RsuMZxDmjNQ==; AWSALBCORS=QEhnWZazMReavGD5fhReZgRFKpJmspiCqASSoNHdRZ1VytpYiix6m7VDR4kQ+LerVz2w5nT2i8c2w7uMasyfeMYtiOv2XriiTUlDCbj5S6DcboWu28zQHfV3j5bfGODPYKi8LAsm+G0jVom+eHR+K1XRXp4OHB9eclZ4h2cJj5QtosvDt2UetHB8wFz2xifE61iVpjMA+v0ivchJ7WltitGpk8EVKo6wy8t+0Q0NcSMJ4QxaLIV5nQV1Lk7xEDX+lK+HB3+RsuMZxDmjNQ==; INGRESSAFFINITY=2265ae43ed32158b',
        'dnt': '1',
        'pragma': 'no-cache',
        'referer': 'https://www.google.com/',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'cross-site',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    response = session.get('https://courses.osu.edu/psp/csosuct/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers)
    print(response)
    print(response.request.headers)
    print(session.cookies.get_dict())

    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'CS-PORTAL-PSJSESSIONID=pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150; AWSALB=7nStDVIbJrwrMRm6Zv7/Xg6M4Z2eDX+VWwxmcbA4kZ2Fbxtk0AXITUTgfoPYIIbtLyvGDHtTFX363jOYzfD3rrazQpVVMqweVZTQZITpBO0XaZ0TspaovvaUhVPY; AWSALBCORS=7nStDVIbJrwrMRm6Zv7/Xg6M4Z2eDX+VWwxmcbA4kZ2Fbxtk0AXITUTgfoPYIIbtLyvGDHtTFX363jOYzfD3rrazQpVVMqweVZTQZITpBO0XaZ0TspaovvaUhVPY',
        'dnt': '1',
        'pragma': 'no-cache',
        'referer': 'https://www.google.com/',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'cross-site',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }
    response = session.get('https://courses.osu.edu/psp/csosuct/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', headers=headers)
    print(response)
    print(response.request.headers)
    print(session.cookies.get_dict())

    new_cookies = session.cookies.get_dict()

    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        'content-type': 'application/x-www-form-urlencoded',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'CS-PORTAL-PSJSESSIONID=pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABTX+NR54BJiiOEGC2/jWk7zoym2om4AAAAFAFNkYXRhYnicHYpLDkBAEETfIE5g5Q7EZ8RaJsNGEJ+1s7iXwynTna6qV+kHSOLIGPkbESZ3rFzsHHjdzST600nqWYTZJhrFN7N+HYPcNlQ0WAp5K61DrpVLXS/tglbqeq1VywfUPw6w; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_TOKENEXPIRE=14_Mar_2024_21:41:30_GMT; PS_LASTSITE=https://courses.erppub.osu.edu/psp/ps/; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3Fpage%3DSSR_CLSRCH_ENTRY%22%20%22label%22%3A%22Criteria%20Entry%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%221%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; AWSALB=eCrateWQ5gaVGwwDuwXet3VlFzy09ZXGMO5krMTzDWRXr7NZUaLoPSh2RgFS33w5NIvnIq3rwvN6zB0c4rCus1YMbAD6stW4rws2tvS6ANYDOiNlVm7sKV/Rixiw; AWSALBCORS=eCrateWQ5gaVGwwDuwXet3VlFzy09ZXGMO5krMTzDWRXr7NZUaLoPSh2RgFS33w5NIvnIq3rwvN6zB0c4rCus1YMbAD6stW4rws2tvS6ANYDOiNlVm7sKV/Rixiw',
        'dnt': '1',
        'origin': 'https://courses.erppub.osu.edu',
        'pragma': 'no-cache',
        'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    temp_cookies = {
        'CS-PORTAL-PSJSESSIONID': new_cookies['CS-PORTAL-PSJSESSIONID'],
        'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
        'PS_TOKEN': new_cookies['PS_TOKEN'],
        'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
        'SignOnDefault': '',
        'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
        'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
        'PS_LASTSITE': 'https://courses.erppub.osu.edu/psp/ps/',
        'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
        'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3Fpage%3DSSR_CLSRCH_ENTRY%22%20%22label%22%3A%22Criteria%20Entry%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%221%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    }

    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=1&ICType=Panel&ICElementNum=0&ICStateNum=6&ICAction=CLASS_SRCH_WRK2_STRM%2435%24&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lYt88fXu5qjzHCMI4Ttu7BucD1LO0ymJyWZWIYNhA%2F4%3D&ICActionPrompt=false&ICBcDomData=C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_INSTITUTION$31$=OSUSI&CLASS_SRCH_WRK2_STRM$35$=1248'

    response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=temp_cookies, headers=headers, data=data)
    print(response)
    print(response.request.headers)
    print(session.cookies.get_dict())

    with open("test1.html", "w") as f:
        f.write(response.text)
    
    headers = {
        'authority': 'courses.erppub.osu.edu',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        'content-type': 'application/x-www-form-urlencoded',
        # Requests sorts cookies= alphabetically
        # 'cookie': 'CS-PORTAL-PSJSESSIONID=pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABTX+NR54BJiiOEGC2/jWk7zoym2om4AAAAFAFNkYXRhYnicHYpLDkBAEETfIE5g5Q7EZ8RaJsNGEJ+1s7iXwynTna6qV+kHSOLIGPkbESZ3rFzsHHjdzST600nqWYTZJhrFN7N+HYPcNlQ0WAp5K61DrpVLXS/tglbqeq1VywfUPw6w; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LASTSITE=https://courses.erppub.osu.edu/psp/ps/; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3Fpage%3DSSR_CLSRCH_ENTRY%22%20%22label%22%3A%22Criteria%20Entry%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%221%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; PS_TOKENEXPIRE=14_Mar_2024_21:42:34_GMT; AWSALB=sXP/XzZtdRzzQXzCb6Oh33ccodHULOLPyS8s2Ll6TV8rm2AvMsh9tsU+Iz3CR010d/Lu1Rn7TPQWqv9HMBkTE60kQVBb6vXRLxqMAQmrZit5y6afEOp6t4nCiFZD; AWSALBCORS=sXP/XzZtdRzzQXzCb6Oh33ccodHULOLPyS8s2Ll6TV8rm2AvMsh9tsU+Iz3CR010d/Lu1Rn7TPQWqv9HMBkTE60kQVBb6vXRLxqMAQmrZit5y6afEOp6t4nCiFZD',
        'dnt': '1',
        'origin': 'https://courses.erppub.osu.edu',
        'pragma': 'no-cache',
        'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }

    data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=1&ICType=Panel&ICElementNum=0&ICStateNum=7&ICAction=DERIVED_CLSRCH_SSR_EXPAND_COLLAPS%24149%24%242&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lYt88fXu5qjzHCMI4Ttu7BucD1LO0ymJyWZWIYNhA%2F4%3D&ICActionPrompt=false&ICBcDomData=C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_STRM$35$=1248'

    response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=new_cookies, headers=headers, data=data)
    print(response)
    print(response.request.headers)
    print(session.cookies.get_dict())

    with open("test2.html", "w") as f:
        f.write(response.text)

finally:
    session.close()


<Response [200]>
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'Connection': 'keep-alive', 'authority': 'courses.osu.edu', 'accept-language': 'en-US,en;q=0.9', 'cache-control': 'no-cache', 'dnt': '1', 'pragma': 'no-cache', 'referer': 'https://www.google.com/', 'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"Windows"', 'sec-fetch-dest': 'document', 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'cross-site', 'sec-fetch-user': '?1', 'upgrade-insecure-requests': '1', 'Cookie': 'AWSALB=QEhnWZazMReavGD5fhReZgRFKpJmspiCqASSoNHdRZ1VytpYiix6m7VDR4kQ+LerVz2w5nT2i8c2w7uMasyfeMYtiOv2XriiTUlDCbj5S6DcboWu28zQHfV3j5bfGODPYKi8LAsm+G0jVom+eHR+K1XRXp4OHB9

In [85]:
pprint(new_cookies)

{'AWSALB': 'yiJzeB0Z18YE5xAI9B+6IcVMyNDWP0Urn5dVFS3QohsNwLPIe0kWc1iPL9K42CAXTt/DPwmt18xGxo+y/5juL3QXMzQCKB6Z1ICt40iYOShB0MKZSjGpuz99sbDOvwgI0s3KFmqoiWQfxz32kJlNhRMV07M71dpYUfPS1Fe4xCZ+Cf7YpORI//uP2298tOEObzh65hZ7oOVRxoCFoBtKxS8y+/uTeMIbjVN2EalqNbThdbmu3YdGbaVUk3xYP/F64pGlnFvcbEaNS9btWw==',
 'AWSALBCORS': 'yiJzeB0Z18YE5xAI9B+6IcVMyNDWP0Urn5dVFS3QohsNwLPIe0kWc1iPL9K42CAXTt/DPwmt18xGxo+y/5juL3QXMzQCKB6Z1ICt40iYOShB0MKZSjGpuz99sbDOvwgI0s3KFmqoiWQfxz32kJlNhRMV07M71dpYUfPS1Fe4xCZ+Cf7YpORI//uP2298tOEObzh65hZ7oOVRxoCFoBtKxS8y+/uTeMIbjVN2EalqNbThdbmu3YdGbaVUk3xYP/F64pGlnFvcbEaNS9btWw==',
 'AWSALBTG': 'Ym/DKQ2SxQSjSWxPDRZlDkcPrmiKPDxFzCdS3Ek7vbs27M6jIsHwuhlsqSQPTM0V3ChjByKtFEUvHhu/NX7bog0Hf6DPXFRJMJ9+45DydN5s7Hh/96QcG0PZGM1vP9+oznMHTpolNYkdBRWsINdUEziUX7V9S3MKsOIBDdKJSW4/L/6t8/4=',
 'AWSALBTGCORS': 'Ym/DKQ2SxQSjSWxPDRZlDkcPrmiKPDxFzCdS3Ek7vbs27M6jIsHwuhlsqSQPTM0V3ChjByKtFEUvHhu/NX7bog0Hf6DPXFRJMJ9+45DydN5s7Hh/96QcG0PZGM1vP9+oznMHTpolNYkdBRWsINdUEziUX7V9S3MKsOIBDdKJSW4/L/6t8/4=',
 'CS-PORTAL-PSJS

In [82]:
headers = {
    'authority': 'courses.osu.edu',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': 'https://www.google.com/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

response = requests.get('https://courses.osu.edu/psp/csosuct/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', headers=headers)
print(response.request.headers['Cookie'])
print(response.cookies.get_dict())

cookies = {
    'CS-PORTAL-PSJSESSIONID': 'pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150',
    'AWSALB': '7nStDVIbJrwrMRm6Zv7/Xg6M4Z2eDX+VWwxmcbA4kZ2Fbxtk0AXITUTgfoPYIIbtLyvGDHtTFX363jOYzfD3rrazQpVVMqweVZTQZITpBO0XaZ0TspaovvaUhVPY',
    'AWSALBCORS': '7nStDVIbJrwrMRm6Zv7/Xg6M4Z2eDX+VWwxmcbA4kZ2Fbxtk0AXITUTgfoPYIIbtLyvGDHtTFX363jOYzfD3rrazQpVVMqweVZTQZITpBO0XaZ0TspaovvaUhVPY',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'CS-PORTAL-PSJSESSIONID=pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150; AWSALB=7nStDVIbJrwrMRm6Zv7/Xg6M4Z2eDX+VWwxmcbA4kZ2Fbxtk0AXITUTgfoPYIIbtLyvGDHtTFX363jOYzfD3rrazQpVVMqweVZTQZITpBO0XaZ0TspaovvaUhVPY; AWSALBCORS=7nStDVIbJrwrMRm6Zv7/Xg6M4Z2eDX+VWwxmcbA4kZ2Fbxtk0AXITUTgfoPYIIbtLyvGDHtTFX363jOYzfD3rrazQpVVMqweVZTQZITpBO0XaZ0TspaovvaUhVPY',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': 'https://www.google.com/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

response = requests.get('https://courses.erppub.osu.edu/psp/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers)



CS-PORTAL-PSJSESSIONID=8VA_B0qQXPYSy5HTrjNs5KR3vrZqCoL5!790274619; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_DEVICEFEATURES=new:1; PS_LASTSITE=https://courses.erppub.osu.edu/psp/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABQq3IPv/48v8G72KHZUFpcu78mzaW0AAAAFAFNkYXRhYXicHYpLDkBAEAVrECewcgfiM8FWZNgI4rN2FvdyOG90p+t1Je8BojAwRvkG/JMOrFzsHDjdzSTz30nsWKTJJhvlN7O6A73SVhRUWDJlLZb/75mLtWhpRN9p1em0fNPXDrA=; PS_TOKENEXPIRE=14_Mar_2024_22:13:46_GMT; PS_TokenSite=https://courses.erppub.osu.edu/psp/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2femployee%2fcamp%2frefresh=list: %3ftab%3dos_facstaff_erp|%3frp%3dos_facstaff_erp|%3ftab%3dremoteunifieddashboard|%3frp%3dremoteunifieddashboard; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; AWSALB=8zRKlHiT3bP4EXhLypIn+I/39K/c1qd4

In [97]:
import requests
import requests
from requests.utils import dict_from_cookiejar

cookies = {
    'INGRESSAFFINITY': '2265ae43ed32158b',
    'AWSALBTG': 'tEA1ONcW7ZaZN4kAeu20ACB/osf7h2qOPieuz9vTpzKcWsn+SLArX5O+7fTTIqQPN8vqauLD7qHo6wYTpPTamg5M30Yh+WR57yn0R2yK6rirZfd9WRNSKzrjBWZjqpFcQkAWeQ3f/roPgRuWKQ25tmoi+GwpULIK+jaQtYDn4NQLhj88uf4=',
    'AWSALBTGCORS': 'tEA1ONcW7ZaZN4kAeu20ACB/osf7h2qOPieuz9vTpzKcWsn+SLArX5O+7fTTIqQPN8vqauLD7qHo6wYTpPTamg5M30Yh+WR57yn0R2yK6rirZfd9WRNSKzrjBWZjqpFcQkAWeQ3f/roPgRuWKQ25tmoi+GwpULIK+jaQtYDn4NQLhj88uf4=',
    'AWSALB': 'P/D9ZZMCa9EJUovu7aRYofEdbRCiS9PW1dKSbbDENP0kGSlFjonYUMXkOA1lzCAHGTBN6NhU2f+bCi7zuI4ewCxYgl9WNXv1Kdq0tdOczcJt3wWne/XXC/JI6NH/4IV3wC1U3R1G/mPUlaUQQJRME0W1dVeX9vzPRneBCE38sD+FlYY84+6t26YidOaiK9gKLO+g9RUzKByD8VK8xFvSzHoK8rF4luBbjnmg3cvdXIPqMfGAQl/Zc3G109Qy4hOdlQaJxz6R+OF4t0vc4w==',
    'AWSALBCORS': 'P/D9ZZMCa9EJUovu7aRYofEdbRCiS9PW1dKSbbDENP0kGSlFjonYUMXkOA1lzCAHGTBN6NhU2f+bCi7zuI4ewCxYgl9WNXv1Kdq0tdOczcJt3wWne/XXC/JI6NH/4IV3wC1U3R1G/mPUlaUQQJRME0W1dVeX9vzPRneBCE38sD+FlYY84+6t26YidOaiK9gKLO+g9RUzKByD8VK8xFvSzHoK8rF4luBbjnmg3cvdXIPqMfGAQl/Zc3G109Qy4hOdlQaJxz6R+OF4t0vc4w==',
}

headers = {
    'authority': 'courses.osu.edu',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'INGRESSAFFINITY=2265ae43ed32158b; AWSALBTG=tEA1ONcW7ZaZN4kAeu20ACB/osf7h2qOPieuz9vTpzKcWsn+SLArX5O+7fTTIqQPN8vqauLD7qHo6wYTpPTamg5M30Yh+WR57yn0R2yK6rirZfd9WRNSKzrjBWZjqpFcQkAWeQ3f/roPgRuWKQ25tmoi+GwpULIK+jaQtYDn4NQLhj88uf4=; AWSALBTGCORS=tEA1ONcW7ZaZN4kAeu20ACB/osf7h2qOPieuz9vTpzKcWsn+SLArX5O+7fTTIqQPN8vqauLD7qHo6wYTpPTamg5M30Yh+WR57yn0R2yK6rirZfd9WRNSKzrjBWZjqpFcQkAWeQ3f/roPgRuWKQ25tmoi+GwpULIK+jaQtYDn4NQLhj88uf4=; AWSALB=P/D9ZZMCa9EJUovu7aRYofEdbRCiS9PW1dKSbbDENP0kGSlFjonYUMXkOA1lzCAHGTBN6NhU2f+bCi7zuI4ewCxYgl9WNXv1Kdq0tdOczcJt3wWne/XXC/JI6NH/4IV3wC1U3R1G/mPUlaUQQJRME0W1dVeX9vzPRneBCE38sD+FlYY84+6t26YidOaiK9gKLO+g9RUzKByD8VK8xFvSzHoK8rF4luBbjnmg3cvdXIPqMfGAQl/Zc3G109Qy4hOdlQaJxz6R+OF4t0vc4w==; AWSALBCORS=P/D9ZZMCa9EJUovu7aRYofEdbRCiS9PW1dKSbbDENP0kGSlFjonYUMXkOA1lzCAHGTBN6NhU2f+bCi7zuI4ewCxYgl9WNXv1Kdq0tdOczcJt3wWne/XXC/JI6NH/4IV3wC1U3R1G/mPUlaUQQJRME0W1dVeX9vzPRneBCE38sD+FlYY84+6t26YidOaiK9gKLO+g9RUzKByD8VK8xFvSzHoK8rF4luBbjnmg3cvdXIPqMfGAQl/Zc3G109Qy4hOdlQaJxz6R+OF4t0vc4w==',
    'dnt': '1',
    'pragma': 'no-cache',
    'referer': 'https://www.google.com/',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

response = requests.get('https://courses.osu.edu/psp/csosuct/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers)
print(response.request.headers['Cookie'])
print(response.cookies.get_dict())


cookie_dict = dict_from_cookiejar(response.request.headers.get('Cookie'))
print(cookie_dict)




cookies = {
    'CS-PORTAL-PSJSESSIONID': 'pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150',
    'PS_LOGINLIST': 'https://courses.erppub.osu.edu/ps',
    'PS_TokenSite': 'https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID',
    'SignOnDefault': '',
    'PS_DEVICEFEATURES': 'width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0',
    'ExpirePage': 'https://courses.erppub.osu.edu/psp/ps/',
    'PS_LASTSITE': 'https://courses.erppub.osu.edu/psp/ps/',
    'ps_theme': 'node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2',
    'psback': '%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3Fpage%3DSSR_CLSRCH_ENTRY%22%20%22label%22%3A%22Criteria%20Entry%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%221%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22',
    'PS_TOKENEXPIRE': '14_Mar_2024_22:30:00_GMT',
    'AWSALB': '3yJZ0RlHj4k7v5z5T8HlnrnELLDG4HqDiKu0vNgBqC7BaflnMPP46m2j04fpB4HkNL+Xt4R8MA5hwIMNie+xF895FIONXkeL2hsMsNTFSeGfkfmbwPI1qx9kImu1',
    'AWSALBCORS': '3yJZ0RlHj4k7v5z5T8HlnrnELLDG4HqDiKu0vNgBqC7BaflnMPP46m2j04fpB4HkNL+Xt4R8MA5hwIMNie+xF895FIONXkeL2hsMsNTFSeGfkfmbwPI1qx9kImu1',
}

headers = {
    'authority': 'courses.erppub.osu.edu',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/x-www-form-urlencoded',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'CS-PORTAL-PSJSESSIONID=pQI-5oHbft7JlwQ_b3vgAwJd3OSVlvWi!697372150; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; PS_DEVICEFEATURES=width:2560 height:1440 pixelratio:1 touch:0 geolocation:1 websockets:1 webworkers:1 datepicker:1 dtpicker:1 timepicker:1 dnd:1 sessionstorage:1 localstorage:1 history:1 canvas:1 svg:1 postmessage:1 hc:0 maf:0; ExpirePage=https://courses.erppub.osu.edu/psp/ps/; PS_LASTSITE=https://courses.erppub.osu.edu/psp/ps/; ps_theme=node:CAMP portal:EMPLOYEE theme_id:OSU_TANGERINE_FL css:DEFAULT_THEME_FLUID css_f:OSU_FLUID_FLUID accessibility:N formfactor:3 piamode:2; psback=%22%22url%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3Fpage%3DSSR_CLSRCH_ENTRY%22%20%22label%22%3A%22Criteria%20Entry%22%20%22origin%22%3A%22PIA%22%20%22layout%22%3A%221%22%20%22refurl%22%3A%22https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsc%2Fps%2FEMPLOYEE%2FPUB%22%22; PS_TOKENEXPIRE=14_Mar_2024_22:30:00_GMT; AWSALB=3yJZ0RlHj4k7v5z5T8HlnrnELLDG4HqDiKu0vNgBqC7BaflnMPP46m2j04fpB4HkNL+Xt4R8MA5hwIMNie+xF895FIONXkeL2hsMsNTFSeGfkfmbwPI1qx9kImu1; AWSALBCORS=3yJZ0RlHj4k7v5z5T8HlnrnELLDG4HqDiKu0vNgBqC7BaflnMPP46m2j04fpB4HkNL+Xt4R8MA5hwIMNie+xF895FIONXkeL2hsMsNTFSeGfkfmbwPI1qx9kImu1',
    'dnt': '1',
    'origin': 'https://courses.erppub.osu.edu',
    'pragma': 'no-cache',
    'referer': 'https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
}

data = 'ICAJAX=1&ICNAVTYPEDROPDOWN=1&ICType=Panel&ICElementNum=0&ICStateNum=14&ICAction=CLASS_SRCH_WRK2_STRM%2435%24&ICModelCancel=0&ICXPos=0&ICYPos=0&ResponsetoDiffFrame=-1&TargetFrameName=None&FacetPath=None&ICFocus=&ICSaveWarningFilter=0&ICChanged=-1&ICSkipPending=0&ICAutoSave=0&ICResubmit=0&ICSID=lYt88fXu5qjzHCMI4Ttu7BucD1LO0ymJyWZWIYNhA%2F4%3D&ICActionPrompt=false&ICBcDomData=C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue*C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue*C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue*C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue*C~UnknownValue~EMPLOYEE~PUB~COMMUNITY_ACCESS.CLASS_SEARCH.GBL~SSR_CLSRCH_ENTRY~Criteria%20Entry~UnknownValue~UnknownValue~https%3A%2F%2Fcourses.erppub.osu.edu%2Fpsp%2Fps%2FEMPLOYEE%2FPUB%2Fc%2FCOMMUNITY_ACCESS.CLASS_SEARCH.GBL%3FPortalKeyStruct%3Dyes~UnknownValue&ICPanelName=&ICFind=&ICAddCount=&ICAppClsData=&CLASS_SRCH_WRK2_INSTITUTION$31$=OSUSI&CLASS_SRCH_WRK2_STRM$35$=1248'

response = requests.post('https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL', cookies=cookies, headers=headers, data=data)


AWSALB=P/D9ZZMCa9EJUovu7aRYofEdbRCiS9PW1dKSbbDENP0kGSlFjonYUMXkOA1lzCAHGTBN6NhU2f+bCi7zuI4ewCxYgl9WNXv1Kdq0tdOczcJt3wWne/XXC/JI6NH/4IV3wC1U3R1G/mPUlaUQQJRME0W1dVeX9vzPRneBCE38sD+FlYY84+6t26YidOaiK9gKLO+g9RUzKByD8VK8xFvSzHoK8rF4luBbjnmg3cvdXIPqMfGAQl/Zc3G109Qy4hOdlQaJxz6R+OF4t0vc4w==; AWSALBCORS=P/D9ZZMCa9EJUovu7aRYofEdbRCiS9PW1dKSbbDENP0kGSlFjonYUMXkOA1lzCAHGTBN6NhU2f+bCi7zuI4ewCxYgl9WNXv1Kdq0tdOczcJt3wWne/XXC/JI6NH/4IV3wC1U3R1G/mPUlaUQQJRME0W1dVeX9vzPRneBCE38sD+FlYY84+6t26YidOaiK9gKLO+g9RUzKByD8VK8xFvSzHoK8rF4luBbjnmg3cvdXIPqMfGAQl/Zc3G109Qy4hOdlQaJxz6R+OF4t0vc4w==; AWSALBTG=tEA1ONcW7ZaZN4kAeu20ACB/osf7h2qOPieuz9vTpzKcWsn+SLArX5O+7fTTIqQPN8vqauLD7qHo6wYTpPTamg5M30Yh+WR57yn0R2yK6rirZfd9WRNSKzrjBWZjqpFcQkAWeQ3f/roPgRuWKQ25tmoi+GwpULIK+jaQtYDn4NQLhj88uf4=; AWSALBTGCORS=tEA1ONcW7ZaZN4kAeu20ACB/osf7h2qOPieuz9vTpzKcWsn+SLArX5O+7fTTIqQPN8vqauLD7qHo6wYTpPTamg5M30Yh+WR57yn0R2yK6rirZfd9WRNSKzrjBWZjqpFcQkAWeQ3f/roPgRuWKQ25tmoi+GwpULIK+jaQtYDn4NQLhj88uf4=; INGRESSAFFINITY=2265ae43ed32158b; CS-POR

AttributeError: 'str' object has no attribute 'value'

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup


In [36]:
def click_Add_Srch_Criteria(driver):
    """Clicks the 'Additional Search Criteria' button on the page to reveal additional options (e.g. lookup by last name)"""
    more_buttons = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//img[@alt='Additional Search Criteria']")))

    if more_buttons.is_displayed():
        more_buttons.click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "SSR_CLSRCH_WRK_LAST_NAME$8")))


def submit_form(driver, url, term, department, last_name, numFails=0):
    """Submits the form on the page and returns the html of the page"""
    webDriverWait = WebDriverWait(driver, 20)
    driver.get(url)

    # selecting spring 2023 and then reopening additional search criteria
    ########################################################
    toWaitOn = webDriverWait.until(
        EC.visibility_of_element_located((By.NAME, "CLASS_SRCH_WRK2_STRM$35$")))
    # toWaitOn = driver.find_element(By.ID, "SSR_CLSRCH_WRK_LAST_NAME$8")

    select = Select(driver.find_element(By.NAME, "CLASS_SRCH_WRK2_STRM$35$"))

    if select.first_selected_option.get_attribute("value") != str(term):
        select.select_by_value(str(term))  # change this value for different semesters
        webDriverWait.until(EC.staleness_of(toWaitOn))
    
    click_Add_Srch_Criteria(driver)

    # lastname search, type = text
    lastNameSearch = webDriverWait.until(
        EC.visibility_of_element_located((By.NAME, "SSR_CLSRCH_WRK_LAST_NAME$8")))
    lastNameSearch.send_keys(last_name)

    foundElement = webDriverWait.until(
        EC.visibility_of_element_located(
        (By.NAME, "SSR_CLSRCH_WRK_SSR_EXACT_MATCH2$8")))
    select = Select(foundElement)
    select.select_by_value("E")  # exactly the last name, not just begins with
    # driver.find_element(By.NAME, "SSR_CLSRCH_WRK_LAST_NAME$8").send_keys(lastName)

    # major selection (CSE), type = select
    foundElement = webDriverWait.until(
        EC.visibility_of_element_located(
        (By.NAME, "SSR_CLSRCH_WRK_SUBJECT_SRCH$1")))
    select = Select(foundElement)
    # select.select_by_visible_text("Computer Science & Engineering")
    select.select_by_value(
        department)  # DONT KNOW WHY THIS DOESN'T PROPERLY SELECT CSE
    # webDriverWait.until(EC.element_attribute_to_include((By.XPATH, "//option[@value='CSE']"), "selected"))


    ########################################################

    # final clicks (unchecking open classes only checkbox and clicking search)
    ########################################################
    try:
        webDriverWait.until(
        EC.element_to_be_clickable(
            (By.NAME, "SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4"))).click()
    except ElementClickInterceptedException:
        if numFails == 0:
            print("Element is not clickable... Trying again")
            submit_form(driver=driver,
                        url=url,
                        term=term,
                        department=department,
                        last_name=last_name,
                        numFails=1)
        else:
            return None

    # driver.find_element(By.NAME, "SSR_CLSRCH_WRK_SSR_OPEN_ONLY$4").click()
    webDriverWait.until(
        EC.text_to_be_present_in_element_value(
        (By.NAME, "SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4"), "N"))

    webDriverWait.until(
        EC.element_to_be_clickable(
        (By.NAME, "CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH"))).click()

    # looking for successful search OR an error
    successOrFail = webDriverWait.until(
        EC.any_of(
        EC.presence_of_element_located((By.ID, "DERIVED_CLSMSG_ERROR_TEXT")),
        EC.presence_of_element_located((By.CLASS_NAME, "PSLEVEL1GRIDNBONBO"))))
    # test = driver.find_element(By.ID, "DERIVED_CLSMSG_ERROR_TEXT")
    # test.accessible_name
    # test.text
    # test.get_attribute("id")
    # test.find_element(By.XPATH, "//option[value=UGRD").get_property("selected")
    # print(successOrFail.get_attribute("id"))
    if successOrFail.get_attribute("id") == "DERIVED_CLSMSG_ERROR_TEXT":
        print(last_name + ": " + successOrFail.text)
        return None
    else:
        print(last_name + ": Search Successful")
        return driver.page_source
    
def parse_form(soup: BeautifulSoup):
    """Parses the html of the form and returns a list of courses"""

    print("Parsing form")
    # with open("test.html", "w") as f:
    #     f.write(soup.prettify())

    availabilityDict = {
        "/cs/ps/cache85921/PS_CS_STATUS_OPEN_ICN_1.gif": "Available",
        "/cs/ps/cache85921/PS_CS_STATUS_WAITLIST_ICN_1.JPG": "Waitlist",
        "/cs/ps/cache85921/PS_CS_STATUS_CLOSED_ICN_1.gif": "Closed"
    }

    coursesRows = soup.find("table", {"id": "ACE_$ICField$4$$0"})
    coursesRows = coursesRows.find("tbody", recursive=False).find_all("tr", recursive=False) # type: ignore
    # print(len(coursesRows))
    for row in coursesRows:
        # this find keeps making the program crash
        fullCourseTitleElement = row.find("td", {"class": "PAGROUPBOXLABELLEVEL1"})
        if fullCourseTitleElement is None:
            continue

        fullCourseTitle = fullCourseTitleElement.get_text().strip()
        # print(fullCourseTitle)
        splitTitle = fullCourseTitle.split("-", maxsplit=1)
        deptAndNumber = splitTitle[0].strip()
        dept, number = deptAndNumber.split(" ", maxsplit=1)

        sectionRows = row.find_all(id=lambda x: x and x.startswith("ACE_SSR_CLSRSLT_WRK_GROUPBOX3"))

        # print(len(sectionRows))
        for sectionRow in sectionRows:
            rowData = sectionRow.find_all("td", {"class": "PSLEVEL3GRIDODDROW"})
            section_id = rowData[0].text.strip()
            section_info = rowData[1].text.strip().replace("\n", ",")
            days_and_times = rowData[2].text.strip()
            room = rowData[3].text.strip()


            instructors = rowData[4].text.replace("\n", "").split(",")
            if instructors[0].lower() != "to be announced":
                for instructor in instructors:
                    print("will save this instructor", instructor.split(" ")[0], instructor.split(" ")[-1])
            else:
                print("bruh - will save queryset", instructors[0])

            meeting_dates = rowData[5].text.strip().split("\n")
            meeting_dates = meeting_dates[0].strip()
            # only get the first two for start and end date
            dates = meeting_dates.split(" - ")
            start_date = dates[0].strip()
            end_date = dates[1].strip()
            # print(section_id, section_info, days_and_times, room, instructors, start_date, end_date)


In [37]:
# selenium setup
# print("test1")
options = webdriver.ChromeOptions()
# # options.binary_location = os.environ.get("GOOGLE_CHROME_BIN") # type: ignore
options.add_argument('--no-sandbox')
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--headless')
options.add_argument('--disable-gpu')

# driver = webdriver.Chrome(executable_path=os.environ.get("CHROMEDRIVER_PATH"), chrome_options=options) # type: ignore
# print("test2")
# install chromedriver if not already installed
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

try:
    url = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL?PortalActualURL=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2fEMPLOYEE%2fPUB%2fc%2fCOMMUNITY_ACCESS.CLASS_SEARCH.GBL&PortalRegistryName=EMPLOYEE&PortalServletURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsp%2fps%2f&PortalURI=https%3a%2f%2fcourses.erppub.osu.edu%2fpsc%2fps%2f&PortalHostNode=CAMP&NoCrumbs=yes&PortalKeyStruct=yes"

    last_name = "bucci"
    term = 1248
    department = "CSE"
    submittedForm = submit_form(driver, url, term, department, last_name, numFails=0)
    if submittedForm is not None:
        soup = BeautifulSoup(submittedForm, 'html.parser')
        parse_form(soup)
finally:
    driver.quit()


C:\Users\Hanson Ma\AppData\Local\Temp\ipykernel_17768\3625136289.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


bucci: Search Successful
Parsing form
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announced
bruh - will save queryset To be Announ

In [9]:
import requests
from bs4 import BeautifulSoup
import json

In [69]:
def send_request(action, info, cookie, session, num=0):
    link = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL"

    data = {
        "ICAction": action,
        "ICSID": cookie['sessionId'],
    }

    if action == "CLASS_SRCH_WRK2_STRM$35$":
        data["CLASS_SRCH_WRK2_STRM$35$"] = str(info['term'])
    # elif action == "SSR_CLSRCH_WRK_CAMPUS":
    #     data["CLASS_SRCH_WRK2_STRM$35$"] = str(info['term'])
    #     data["SSR_CLSRCH_WRK_CAMPUS$0"] = info['campus']
    elif action == "DERIVED_CLSRCH_SSR_EXPAND_COLLAPS$149$$2":
        data["CLASS_SRCH_WRK2_STRM$35$"] = str(info['term'])
        # data["SSR_CLSRCH_WRK_CAMPUS$0"] = info['campus']
        # data["SSR_CLSRCH_WRK_CATALOG_NBR$2"] = str(info['courseNumber'])
        data["SSR_CLSRCH_WRK_SUBJECT_SRCH$1"] = info['department']
        data["SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4"] = "N"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "cookie": cookie['cookie'],
    }

    response = session.post(link, data=data, headers=headers)

    with open(f"test{num}.html", "w") as f:
        f.write(response.text)

    return response.status_code == 200

In [105]:
def result_process(link, info, cookie, session: requests.Session):
    sections = []
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
        "Content-Type": "application/x-www-form-urlencoded",
        "cookie": cookie['cookie'],
    }

    data = {
        "ICAction": "CLASS_SRCH_WRK2_SSR_PB_CLASS_SRCH",
        "ICSID": cookie['sessionId'],
        "SSR_CLSRCH_WRK_LAST_NAME$8": info['lastName'],
        "SSR_CLSRCH_WRK_SSR_EXACT_MATCH2$8": "E",
        "SSR_CLSRCH_WRK_SSR_OPEN_ONLY$chk$4": "N",
    }

    try:
        response = session.post(link, data=data, headers=headers)
        response.raise_for_status()
        with open("test3.html", "w") as f:
            f.write(response.text)
    except requests.exceptions.HTTPError as err:
        print(err)
        return sections, False

    soup = BeautifulSoup(response.text, 'html.parser')
    with open("test.html", "w") as f:
        f.write(soup.prettify())
    # section_divs = soup.select('div[id^=win0divSSR_CLSRSLT_WRK_GROUPBOX3]') 
    # for div in section_divs:
    #     section = {
    #         'sectionNumber': div.select_one('span[id^=MTG_CLASS_NBR]').text, # type: ignore
    #         'sectionName': div.select_one('span[id^=MTG_CLASSNAME]').text.replace('\n', ' '), # type: ignore
    #         'sectionDate': div.select_one('span[id^=MTG_DAYTIME]').text, # type: ignore
    #         'sectionRoom': div.select_one('span[id^=MTG_ROOM]').text, # type: ignore
    #         'meetingDates': div.select_one('span[id^=MTG_TOPIC]').text, # type: ignore
    #         'sectionInstructor': info['fullName'],
    #         'sectionAvailability': div.select_one('div[id^=win0divDERIVED_CLSRCH_SSR_STATUS_LONG] > div > img')['alt'], # type: ignore
    #     }
    #     sections.append(section)

    availabilityDict = {
        "/cs/ps/cache85921/PS_CS_STATUS_OPEN_ICN_1.gif": "Available",
        "/cs/ps/cache85921/PS_CS_STATUS_WAITLIST_ICN_1.JPG": "Waitlist",
        "/cs/ps/cache85921/PS_CS_STATUS_CLOSED_ICN_1.gif": "Closed"
    }
    coursesRows = soup.select("div[id^=win0divSSR_CLSRSLT_WRK_GROUPBOX2\$]")
    # with open("testt.html", "w") as f:
    #     f.write(str(coursesRows[0]))
    # # coursesRows = soup.find("table", {"id": "ACE_$ICField$4$$0"})
    # # coursesRows = coursesRows.find("tbody", recursive=False).find_all("tr", recursive=False) # type: ignore
    # coursesRows = coursesRows.find('tbody', recursive=False).find_all('tr', recursive=False) # type: ignore

    # print(len(coursesRows))
    for row in coursesRows:
        # this find keeps making the program crash
        fullCourseTitleElement = row.find("td", {"class": "PAGROUPBOXLABELLEVEL1"})
        fullCourseTitleElement = row.select_one("div[id^=win0divSSR_CLSRSLT_WRK_GROUPBOX2GP]")
        if fullCourseTitleElement is None:
            continue

        fullCourseTitle = fullCourseTitleElement.get_text().strip()
        # print(fullCourseTitle)
        splitTitle = fullCourseTitle.split("-", maxsplit=1)
        deptAndNumber = splitTitle[0].strip()
        dept, number = deptAndNumber.split(" ", maxsplit=1)

        sectionRows = row.select("div[id^=win0divSSR_CLSRSLT_WRK_GROUPBOX3]")

        # print(len(sectionRows))
        for sectionRow in sectionRows:
            rowData = sectionRow.find_all("td", {"class": "PSLEVEL3GRIDODDROW"})
            section_id = rowData[0].text.strip()
            section_info = rowData[1].text.strip().replace("\n", ",")
            days_and_times = rowData[2].text.strip()
            room = rowData[3].text.strip()

            instructors = rowData[4].text.replace("\n", "").split(",")

            meeting_dates = rowData[5].text.strip().split("\n")
            meeting_dates = meeting_dates[0].strip()
            # only get the first two for start and end date
            dates = meeting_dates.split(" - ")
            start_date = dates[0].strip()
            end_date = dates[1].strip()
            # start_date = datetime.strptime(start_date, "%m/%d/%Y").date()
            # end_date = datetime.strptime(end_date, "%m/%d/%Y").date()
            availabilityElement = rowData[6].find("img")
            availability = availabilityDict.get(availabilityElement.get_attribute_list("src")[0], "")

            section = {
                'sectionNumber': section_id,
                'sectionName': section_info,
                'sectionDate': days_and_times,
                'sectionRoom': room,
                'meetingDates': meeting_dates,
                'sectionInstructor': info['fullName'],
                'sectionAvailability': availability,
            }
            sections.append(section)

    found = len(sections) > 0

    # Call addToDatabase function
    # addToDatabase(sections, info)
    # pprint(sections)

    return sections, found

In [108]:
def update_same_subject(info, cookie):
    link = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL"
    session = requests.Session()

    send_request("CLASS_SRCH_WRK2_STRM$35$", info, cookie, session, 1)
    # send_request("SSR_CLSRCH_WRK_CAMPUS", info, cookie, session)
    send_request("DERIVED_CLSRCH_SSR_EXPAND_COLLAPS$149$$2", info, cookie, session, 2)

    result = result_process(link, info, cookie, session)
    session.close()

    # Call result_process function
    return result

In [15]:
def get_basic_info(session: requests.Session):
    link = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL"

    try:
        response = session.post(link)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        return

    cookie_str = "; ".join([f"{c.name}={c.value}" for c in response.cookies])

    soup = BeautifulSoup(response.content, 'html.parser')
    session_id_input = soup.find('input', {'name': 'ICSID'})

    if session_id_input is None:
        print("Unable to read ICSID")
        return

    session_id = session_id_input['value'] # type: ignore

    result = {'cookie': cookie_str, 'sessionId': session_id}
    print(result)
    return result

In [16]:
# test get_basic_info
session = requests.Session()
get_basic_info(session)
session.close()

{'cookie': 'CS-PORTAL-PSJSESSIONID=JxtPTr8VF5maaAQlLL_RvGplCFV3qHmz!-957170355; ExpirePage=https://courses.erppub.osu.edu/psc/ps/; PS_DEVICEFEATURES=new:1; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABSSa98OYkd0QY9B/8E5PKGqUmJKuW4AAAAFAFNkYXRhYnicHYpJDkBAFESfIU5g5Q46tDZsRbARxLB2FvdyOMVPquq/St1AGPiep3x8/ks6Fk42dnrpYhR930HUMwvjVTSILyZtO1qls2RYHKmykOfUvzcYNaXcqTdUaqz4Ey/UxQ60; PS_TOKENEXPIRE=18_Mar_2024_02:05:44_GMT; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; AWSALB=VerFBwD7tIiNzTWgY2qqYlrFUsMEdL9aqi5Sck1Tl0SPEEomx3V21QeNWu6CPLKjmvbWgPzHgH95blgcfT6mYKTCUn/W9gYEBbSg8lSx0OeBmgk7Q63C9gHEzt9W; AWSALBCORS=VerFBwD7tIiNzTWgY2qqYlrFUsMEdL9aqi5Sck1Tl0SPEEomx3V21QeNWu6CPLKjmvbWgPzHgH95blgcfT6mYKTCUn/W9gYEBbSg8lSx0OeBmgk7Q63C9gHEzt9W', 'sessionId': 'iaRjH8gOqqOM7bycEM2yA6gVkmqehUvMDbJZD2PJTlc='}


In [52]:
# test send_request
session = requests.Session()
cookie = get_basic_info(session)
info = {
    'term': 1248,
    'department': 'CSE',
    'courseNumber': 2221,
    'lastName': 'Bucci',
    'fullName': 'Paulo Bucci'
}
send_request("CLASS_SRCH_WRK2_STRM$35$", info, cookie, session, 1)
send_request("DERIVED_CLSRCH_SSR_EXPAND_COLLAPS$149$$2", info, cookie, session, 2)
session.close()

{'cookie': 'CS-PORTAL-PSJSESSIONID=_WZPdfpXae9As-zWsURvOdGrxhi2zRPA!929891169; ExpirePage=https://courses.erppub.osu.edu/psc/ps/; PS_DEVICEFEATURES=new:1; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rQAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABRmGhkiTpYkDAtPT3GGm1zpTqU1Am0AAAAFAFNkYXRhYXicHYpLDkBAEAXLJ05g5Q6EMcJWZNgI4rOes7iXw3l08up1dfoG4igMAvUT8k82sHKxc+AUzyT7tpPEsUjTTTbKPbN+B3q1NZQYLLm6Fivanx2Frh9rGtGoK8UqvNT/DrA=; PS_TOKENEXPIRE=18_Mar_2024_02:48:35_GMT; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; AWSALB=TCXYfzrqqfO/e2zyEepN7EMR34FPRAssbbHX7W4GcnPRimae53HjYTTbcpXWA8zKmtJKrmGiZJpFK7MXHp1dsZtbVBnmkkXZtqs+pj4vhU1sXzNmI2ily/Z2DjOp; AWSALBCORS=TCXYfzrqqfO/e2zyEepN7EMR34FPRAssbbHX7W4GcnPRimae53HjYTTbcpXWA8zKmtJKrmGiZJpFK7MXHp1dsZtbVBnmkkXZtqs+pj4vhU1sXzNmI2ily/Z2DjOp', 'sessionId': '98jaBquTlPxb088fXWxoJjvNAeRuCugOQ57dZzWthOg='}


In [53]:
# test result_process
session1 = requests.Session()
cookie = get_basic_info(session1)
info = {
    'term': 1248,
    'department': 'CSE',
    'courseNumber': 2221,
    'lastName': 'Bucci',
    'fullName': 'Paulo Bucci'
}

link = "https://courses.erppub.osu.edu/psc/ps/EMPLOYEE/PUB/c/COMMUNITY_ACCESS.CLASS_SEARCH.GBL"

session2 = requests.Session()
send_request("CLASS_SRCH_WRK2_STRM$35$", info, cookie, session2, 1)
send_request("DERIVED_CLSRCH_SSR_EXPAND_COLLAPS$149$$2", info, cookie, session2, 2)

# Call result_process function
print(result_process(link, info, cookie))
session1.close()
session2.close()

{'cookie': 'CS-PORTAL-PSJSESSIONID=pthPdkOYB_WcNiYQJPhGhmGgN758ephM!929891169; ExpirePage=https://courses.erppub.osu.edu/psc/ps/; PS_DEVICEFEATURES=new:1; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABQMLHxpWYrAXpwOWHEyLZqPLZstr24AAAAFAFNkYXRhYnicHYpJDkBQEESfIU5g5Q6Eb16KYCOIYe0s7uVwyu+kqvp19QP4nus4ytfFTtSzcrFzMEg3k+jfToKBRRhuolF8M+u3p1MWhhRDQazM5Rm19YZE199LZaKuVVNJBj7Vug7D; PS_TOKENEXPIRE=18_Mar_2024_02:48:54_GMT; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; AWSALB=I/WZmHr5aPGoScw5R7+Y3u3Aij3iG0cAo6A9uEif+7qne5J4Q0Zn8BU2cLnjc8uR+qL0ywD+YeH7wVQaaUK1Bn+3wl5addsIus6kd5NbJmm1B4HXj3kQechRZC4S; AWSALBCORS=I/WZmHr5aPGoScw5R7+Y3u3Aij3iG0cAo6A9uEif+7qne5J4Q0Zn8BU2cLnjc8uR+qL0ywD+YeH7wVQaaUK1Bn+3wl5addsIus6kd5NbJmm1B4HXj3kQechRZC4S', 'sessionId': 'nbt2R2thv3KttFCz+P6Z3kQvtyogogOWMmZJF0ETmBE='}
[{'sectionNumber': '6934', 'sectionNa

In [113]:
def get_department(chosen, term, campus, course_number):
    to_return = []
    try:
        with open('search/fixtures/search/instructors.json', 'r') as json_file:
            all_instructors = json.load(json_file)
    except FileNotFoundError as err:
        print(err)
        return


    instructors = []
    for instructor in all_instructors:
        # print(instructor['fields']['department'], chosen)
        if instructor['fields']['department'][0] == chosen:
            instructors.append(instructor)

    # instructors = [
    #     {
    #         "model": "search.instructor",
    #         "pk": 2388,
    #         "fields": {
    #             "first_name": "Paolo",
    #             "last_name": "Bucci",
    #             "department": [
    #                 "CSE"
    #             ]
    #         }
    #     },
    #     {
    #         "model": "search.instructor",
    #         "pk": 2389,
    #         "fields": {
    #             "first_name": "Wayne",
    #             "last_name": "Heym",
    #             "department": [
    #                 "CSE"
    #             ]
    #         }
    #     },
    #     {
    #         "model": "search.instructor",
    #         "pk": 2390,
    #         "fields": {
    #             "first_name": "Nickalaus",
    #             "last_name": "Painter",
    #             "department": [
    #                 "CSE"
    #             ]
    #         }
    #     }
    # ]
    # print(instructors)

    # time the searches
    import time
    start = time.time()
    
    for instructor in instructors:
        for dept in instructor['fields']['department']:
            if dept == chosen:
                session = requests.Session()
                cookie = get_basic_info(session)
                info = {
                    'lastName': instructor['fields']['last_name'],
                    'term': term,
                    'campus': campus,
                    'courseNumber': course_number,
                    'fullName': instructor['fields']['first_name'] + " " + instructor['fields']['last_name'],
                    'courseFullName': chosen + " " + str(course_number),
                    'department': chosen,
                }
                print(info)

                # Call update_same_subject function
                to_return.append(update_same_subject(info, cookie))
                session.close()
    session.close()

    end = time.time()
    return [i[0] for i in to_return], end - start

In [114]:
sections, time = get_department("CSE", 1248, "COL", 2221)
pprint(sections)
print(time)

{'cookie': 'CS-PORTAL-PSJSESSIONID=padPzNpyOzugAVMJSTfP5Vyz5gsWgDU7!-957170355; ExpirePage=https://courses.erppub.osu.edu/psc/ps/; PS_DEVICEFEATURES=new:1; PS_LASTSITE=https://courses.erppub.osu.edu/psc/ps/; PS_LOGINLIST=https://courses.erppub.osu.edu/ps; PS_TOKEN=rgAAAAQDAgEBAAAAvAIAAAAAAAAsAAAABABTaGRyAk4AgQg4AC4AMQAwABQJgDt1x1p1k9+D0awrLsMpt0AeQW4AAAAFAFNkYXRhYnicHYpLDkBAEAVrECewcgfiF8xSZNgI4rN2FvdyOI9Oul5X591A4HvGKB+Pf+KehZONHae9GGXfdRA6Zmm0ygb5xaRuT6esCjIKKhJlKeY04vdNxfJnK9b6tFg1LbzUPw64; PS_TOKENEXPIRE=18_Mar_2024_04:23:29_GMT; PS_TokenSite=https://courses.erppub.osu.edu/psc/ps/?CS-PORTAL-PSJSESSIONID; SignOnDefault=; AWSALB=C1/oveqFN/uJN1G809exkzPCzzvJNZve3XvfnGOc3AEIYIBqwAr3k728gDrei+n3d357iCKD1hNz8EU72iaOKbFQlMw/lCa5SPkCV/OBh8bV8f6Z/LzA8FsCq3L2; AWSALBCORS=C1/oveqFN/uJN1G809exkzPCzzvJNZve3XvfnGOc3AEIYIBqwAr3k728gDrei+n3d357iCKD1hNz8EU72iaOKbFQlMw/lCa5SPkCV/OBh8bV8f6Z/LzA8FsCq3L2', 'sessionId': '+L/l5A8x4eOIjEgWDpGuDCX5KVpHOY6ybIrQ4PqDags='}
{'lastName': 'Mallon', 'term': 1248,